# Код для расширения словаря Полякова

In [1]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm
from collections import Counter
os.chdir('C:\\Users\\Arseny\\Desktop\\repository\\ocs\\dictionary')

## Мой старый код для порождения словоформ леммы

In [2]:
paradigm_types = {
'N' : 'sg,nom/acc	sg,acc	sg,gen/acc	sg,dat	sg,loc	sg,ins	sg,voc	pl,nom	pl,acc	pl,gen/acc	pl,dat	pl,loc	pl,ins	du,nom/acc	du,gen/loc	du,dat/ins',
'A1' : 'plen,sg,m,nom/acc	brev,sg,m,nom/acc	brev,sg,n,nom/acc	brev,sg,m/n,gen/acc	brev,sg,m/n,dat	brev,sg,m/n,loc	plen/brev,sg,m/n,ins	brev,sg,f,nom	brev,sg,f,acc	brev,sg,f,nom	brev,sg,f,dat/loc	plen/brev,sg,f,ins	brev,pl,m,nom	brev,pl,m,acc|brev,pl,f,nom/acc	brev,pl,n,nom/acc	plen/brev,pl,gen/loc	plen/brev,pl,dat	plen/brev,pl,ins	brev,du,m,nom/acc	brev,du,n/f,nom/acc	brev,du,gen/loc	brev,du,dat/ins	plen,sg,m,nom/acc	plen,sg,n,nom/acc	plen,sg,m/n,gen	plen,sg,m,acc	plen,sg,m/n,dat	plen,sg,m/n,loc	plen,sg,f,nom	plen,sg,f,acc	plen,sg,f,gen	plen,sg,f,dat/loc	plen,pl,m,nom	plen,pl,m,acc|plen,pl,f,nom/acc	plen,pl,n,nom/acc	plen,du,m,nom/acc	plen,du,n/f,nom/acc	plen,du,gen/loc	plen/brev,du,dat/ins	comp1	comp2	brev,sg,m,voc	plen,pl,m,nom1',
'A2' : 'brev,sg,m,nom/acc	brev,sg,m,nom/acc	brev,sg,n,nom/acc	brev,sg,m/n,gen/acc	brev,sg,m/n,dat	brev,sg,m/n,loc	brev,sg,m/n,ins	brev,sg,f,nom	brev,sg,f,acc	brev,sg,f,nom	brev,sg,f,dat/loc	brev,sg,f,ins	brev,pl,m,nom	brev,pl,m,acc|brev,pl,f,nom/acc	brev,pl,n,nom/acc	brev,pl,gen/loc	brev,pl,dat	brev,pl,ins	brev,du,m,nom/acc	brev,du,n/f,nom/acc	brev,du,gen/loc	brev,du,dat/ins',
'PA1' : 'brev,sg,m,nom/acc	brev,sg,n,nom/acc	brev,sg,m/n,gen	brev,sg,m,acc	plen/brev,sg,m/n,dat	plen/brev,sg,m/n,loc	brev,sg,m/n,ins	brev,sg,f,nom	brev,sg,f,acc	brev,sg,f,gen	plen/brev,sg,f,dat/loc	plen/brev,sg,f,ins	brev,pl,m,nom	brev,pl,m,acc|brev,pl,f,nom/acc	brev,pl,n,nom/acc	brev,pl,gen/loc	brev,pl,dat	brev,pl,ins	brev,du,m,nom/acc	brev,du,n/f,nom/acc	brev,du,gen/loc	brev,du,dat/ins	plen,sg,m,nom/acc	plen,sg,n,nom/acc	plen,sg,m/n,gen	plen,sg,m,acc	-	plen,sg,m/n,ins	plen,sg,f,nom	plen,sg,f,acc	plen,sg,f,gen	plen,sg,f,dat/loc	plen,pl,m,nom	plen,pl,m,acc|plen,pl,f,nom/acc	plen,pl,n,nom/acc	plen,pl,gen/loc	plen,pl,dat	plen,pl,ins	plen,du,m,nom/acc	plen,du,n/f,nom/acc	plen,du,gen/loc	plen,du,dat/ins',
'PA2' : 'brev,sg,m,nom/acc	brev,sg,n,nom/acc	brev,sg,m/n,gen	brev,sg,m,acc	brev,sg,m/n,dat	brev,sg,m/n,loc	brev,sg,m/n,ins	brev,sg,f,nom	brev,sg,f,acc	brev,sg,f,gen	brev,sg,f,dat/loc	brev,sg,f,ins	brev,pl,m,nom	brev,pl,m,acc|brev,pl,f,nom/acc	brev,pl,n,nom/acc	brev,pl,gen/loc	brev,pl,dat	brev,pl,ins	brev,du,m,nom/acc	brev,du,n/f,nom/acc	brev,du,gen/loc	brev,du,dat/ins',
'PA2i' : 'sg,m,nom	sg,n,nom/acc	sg,m/n,gen	sg,m,acc	sg,m/n,dat	sg,m/n,loc	sg,m/n,ins	sg,f,nom	sg,f,acc	sg,f,gen	sg,f,dat/loc	sg,f,ins	pl,m/n,nom	pl,m,acc|pl,f,nom/acc	pl,n,acc	pl,gen/loc	pl,dat	pl,ins	du,nom	du,acc	du,gen/loc	du,dat/ins',
'PN' : 'nom	gen	acc	dat	loc	ins	acc,clit	dat,clit	du,nom	du,acc	du,gen/loc	du,dat/ins',
'NUM2' : 'm,nom/acc	n/f,nom/acc	gen/loc	dat/ins',
'NUM3' : 'nom/acc	gen/loc	dat	ins',
'#NUM' : 'sg,nom/acc	sg,gen	sg,dat	sg,loc	sg,ins	pl,nom/acc	pl,gen	pl,loc	pl,dat	pl,ins	du,nom/acc	du,gen/loc	du,dat/ins',
'NUMoe' : 'nom/acc	gen/loc	dat	ins	m/f,nom/acc	n,nom/acc',
'A0' : 'sg,m	sg,f	sg,n	pl	du,m	du,n/f',
'V0' : 'm?	f?',
'V' : 'inf	indic,praes,sg,1p	indic,praes,sg,2p	indic,praes,sg,3p	indic,praes,pl,1p	indic,praes,pl,2p	indic,praes,pl,3p	indic,praes,du,1p	indic,praes,du,2p/3p	imper,sg,2p/3p	imper,pl,2p	imper,pl,1p	imper,du,1p	imper,du,2p	indic,imperf,sg,1p	indic,imperf,sg,2p/3p	indic,imperf,pl,1p	indic,imperf,pl,2p	indic,imperf,pl,3p	indic,imperf,du,1p	indic,imperf,du,2p/3p	indic,aor,sg,1p	indic,aor,sg,2p/3p	indic,aor,pl,1p	indic,aor,pl,2p	indic,aor,pl,3p	indic,aor,du,1p	indic,aor,du,2p/3p	partcp,perf	partcp,perf,sg,m	partcp,praes,act,brev,sg,m/n,nom	partcp,praes,act	partcp,praet,act	partcp,praes,pass	partcp,praet,pass'
}

paradigms = {
'N1t' : {
    'type' : 'N',
    'core' : r'(.*)()()(ъ)',
    'inflex' : '',
    'flex' : 'ъ	=nom/=gen	а	у/ови	ѣ	омъ	е	и/ове	ы/=gen	ѡвъ/ъ^	ѡмъ/амъ	ѣхъ/ахъ	ы^/ми/ами	а^	у^	ома'
    },
'N1t*-a' : {
    'type' : 'N',
    'core' : r'(.*)(е)(л|н|т|к)(ъ)',
    'inflex' : '|е',
    'flex' : '2ъ	=nom/=gen	а	у/ови	ѣ	омъ	е	и/ове	ы/=gen	ѡвъ/ъ^	ѡмъ/амъ	ѣхъ/ахъ	ы^/ми/ами	а^	у^	ома'
    },
'N1t*-b' : {
    'type' : 'N',
    'core' : r'(.*)(е)(в|д)(ъ)',
    'inflex' : 'ь|е',
    'flex' : '2ъ	=nom/=gen	а	у/ови	ѣ	омъ	е	и/ове	ы/=gen	ѡвъ/ъ^	ѡмъ/амъ	ѣхъ/ахъ	ы^/ми/ами	а^	у^	ома'
    },
'N1t*-c' : {
    'type' : 'N',
    'core' : r'(.*)(о)(б|н|т|в|л|м)(ъ)',
    'inflex' : '|о',
    'flex' : '2ъ	=nom/=gen	а	у/ови	ѣ	омъ	е	и/ове	ы/=gen	ѡвъ/ъ^	ѡмъ/амъ	ѣхъ/ахъ	ы^/ми/ами	а^	у^	ома'
    },
'N1j' : {
	'type' : 'N',
    'core' : r'(.*)()()(ь)',
    'inflex' : '',
    'flex' : 'ь	=nom/=gen	я	ю/еви	и/ѣ	емъ	ю	и/еве/іе/е/є	и/=gen	ей/ь^	ємъ/ямъ	ехъ/ѣхъ/яхъ	и^/ьми/ми/ями	я^	ю^	ема'
    },
'N1j*-a' : {
	'type' : 'N',
	'core' : r'(.*)(е)(л|н)(ь)',
	'inflex' : '|е',
	'flex' : '2ь	=nom/=gen	я	ю/еви	и/ѣ	емъ	ю	и/еве/іе	и/=gen	ей/ь^	ємъ/ямъ	ехъ/ѣхъ/яхъ	и^/ьми/ми/ями	я^	ю^	ема'
	},
'N1j*-b' : {
	'type' : 'N',
	'core' : r'(.*)(о)(т|р|л|н)(ь)',
	'inflex' : '|о',
	'flex' : '2ь	=nom/=gen	я	ю/еви	и/ѣ	емъ	ю	и/еве/іе	и/=gen	ей/ь^	ємъ/ямъ	ехъ/ѣхъ/яхъ	и^/ьми/ми/ями	я^	ю^	ема'
	},
'N1je' : {
	'type' : 'N',
	'core' : r'(.*)()(л|р|н)(ь)',
	'inflex' : '',
	'flex' : 'ь	=nom/=gen	я	ю/еви	и/ѣ	емъ	ю	и/еве/іе/е/є	и/=gen	ей	ємъ/ямъ	ехъ/ѣхъ/яхъ	и^/ьми/ми/ями	я^	ю^	ема'
	},
'N1k' : {
	'type' : 'N',
	'core' : r'(.*)(к)()(ъ)',
	'inflex' : 'к|ц|ч',
	'flex' : 'ъ	=nom/=gen	а	у/ови	2ѣ/ѣ	омъ	3е/е	2ы/2ы^	и/=gen	ѡвъ/ъ^	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1g' : {
	'type' : 'N',
	'core' : r'(.*)(г)()(ъ)',
	'inflex' : 'г|з|ж',
	'flex' : 'ъ	=nom/=gen	а	у/ови	2ѣ/ѣ	омъ	3е/е	2и	и/=gen	ѡвъ/ъ^	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1h' : {
	'type' : 'N',
	'core' : r'(.*)(х)()(ъ)',
	'inflex' : 'х|с|ш',
	'flex' : 'ъ	=nom/=gen	а	у/ови	2ѣ/ѣ	омъ	3е/е	2и	и/=gen	ѡвъ/ъ^	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1k*-a' : {
	'type' : 'N',
	'core' : r'(.*)(ок)()(ъ)',
	'inflex' : 'к|ок|ц|ч',
	'flex' : '2ъ	=nom/=gen	а	у/ови	3ѣ/ѣ	омъ	4е	3ы	и/=gen	ѡвъ/2ъ^	ѡмъ/амъ	3ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1k*-b' : {
	'type' : 'N',
	'core' : r'(.*)(ек)()(ъ)',
	'inflex' : 'ьк|ек|ьц|ьч',
	'flex' : '2ъ	=nom/=gen	а	у/ови	3ѣ/ѣ	омъ	4е	3ы	и/=gen	ѡвъ/2ъ^	ѡмъ/амъ	3ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1k*-c' : {
	'type' : 'N',
	'core' : r'(.*)(ек)()(ъ)',
	'inflex' : 'йк|ек|йц|йч',
	'flex' : '2ъ	=nom/=gen	а	у/ови	3ѣ/ѣ	омъ	4е	3ы	и/=gen	ѡвъ/2ъ^	ѡмъ/амъ	3ѣхъ/ахъ	и^/ами	а^	у^	ома'
	},
'N1s' : {
	'type' : 'N',
	'core' : r'(.*)()(ш|ж|ч)(ъ)',
	'inflex' : '',
	'flex' : 'ъ	=nom/=gen	а	у/еви	и/ѣ	емъ	у	и/іе/еве	ы/=gen	ей/ъ^	ємъ/амъ	ехъ/ахъ	ы^/ми/ами	а^	у^	ема'
},
'N1sj' : {
	'type' : 'N',
	'core' : r'(.*)()(ч|щ)(ь)',
	'inflex' : '',
	'flex' : 'ь	=nom/=gen	а	у/еви	и/ѣ	емъ	у	и/іе/еве	и/=gen	ей/ь^	ємъ/амъ	ехъ/ахъ	и^/ьми/ми/ами	а^	у^	ема'
},
'N1c' : {
	'type' : 'N',
	'core' : r'(.*)(ц)()(ъ)',
	'inflex' : 'ц|ч',
	'flex' : 'ъ	=nom/=gen	а	у/еви	ѣ	емъ	2е	ы	ы/=gen	євъ/ѡвъ/ъ^/ей	ємъ/амъ	ѣхъ/ахъ	ы^/ами	а^	у^	ема'
	},
'N1c*-a' : {
	'type' : 'N',
	'core' : r'(.*)(ец)()(ъ)',
	'inflex' : 'ц|ец|ч',
	'flex' : '2ъ	=nom/=gen	а	у/еви	ѣ	емъ	3е	ы	ы/=gen	євъ/ѡвъ/ъ^/ей	ємъ/амъ	ѣхъ/ахъ	ы^/ами	а^	у^	ема'
	},
'N1c*-b' : {
	'type' : 'N',
	'core' : r'(.*)(ец)()(ъ)',
	'inflex' : 'йц|ец|йч',
	'flex' : '2ъ	=nom/=gen	а	у/еви	ѣ	емъ	3е	ы	ы/=gen	євъ/ѡвъ/ъ^/ей	ємъ/амъ	ѣхъ/ахъ	ы^/ами	а^	у^	ема'
	},
'N1c*-c' : {
	'type' : 'N',
	'core' : r'(.*)(ец)()(ъ)',
	'inflex' : 'ьц|ец|ьч',
	'flex' : '2ъ	=nom/=gen	а	у/еви	ѣ	емъ	3е	ы	ы/=gen	євъ/ѡвъ/ъ^/ей	ємъ/амъ	ѣхъ/ахъ	ы^/ами	а^	у^	ема'
	},
'N1a' : {
	'type' : 'N',
	'core' : r'(.*)()()(й)',
	'inflex' : '',
	'flex' : 'й	=nom/=gen	я	ю/еви	и/ѣ	емъ	ю/е	и^/еве/є	и^/=gen	євъ/й^	ємъ/ямъ	ехъ/яхъ	и^/йми/ями	я^	ю^	ема'
	},
'N1i' : {
	'type' : 'N',
	'core' : r'(.*)()()(й)',
	'inflex' : '',
	'flex' : 'й	=nom/=gen	а/я	ю/еви	и	емъ	е	и^/еве/є	и^/=gen	євъ/ѡвъ	ємъ/ѡмъ/ямъ	ехъ/ахъ/яхъ	и^/йми/ями	я^/а^	ю^	ема'
},
'N1e' : {
	'type' : 'N',
	'core' : r'(.*)()()(й)',
	'inflex' : '',
	'flex' : 'й	=nom/=gen	а/я	ю/еви	и	емъ	е	и^/еве/є	и^/=gen	євъ/ѡвъ	ємъ/ѡмъ/ямъ	ехъ/ахъ/яхъ	и^/йми/ями	я^/а^	ю^	ема'
},
'N1in' : {
	'type' : 'N',
	'core' : r'(.*)()()(инъ)',
	'inflex' : '',
	'flex' : 'инъ	=nom/=gen	ина	ину	инѣ	иномъ	ине	е	ы/=gen	ъ/ѡвъ	ѡмъ/амъ	ѣхъ	ы^	а^	у^	ома'
},
#____________________________________________________________________________________________________________________________________
'N2t' : {
	'type' : 'N',
	'core' : r'(.*)()()(о)',
	'inflex' : '',
	'flex' : 'о	=nom	а	у	ѣ	омъ	=nom	а^	=nom	ъ	ѡмъ/амъ	ѣхъ/ахъ	ы/ами	ѣ^	у^	ома'
},
'N2t*-a' : {
	'type' : 'N',
	'core' : r'(.*)()(н|л)(о)',
	'inflex' : '|о',
	'flex' : 'о	=nom	а	у	ѣ	омъ	=nom	а^	=nom	2ъ	ѡмъ/амъ	ѣхъ/ахъ	ы/ами	ѣ^	у^	ома'
},
'N2t*-b' : {
	'type' : 'N',
	'core' : r'(.*)()(н|л)(о)',
	'inflex' : '|е',
	'flex' : 'о	=nom	а	у	ѣ	омъ	=nom	а^	=nom	2ъ	ѡмъ/амъ	ѣхъ/ахъ	ы/ами	ѣ^	у^	ома'
},
'N2t*-c' : {
	'type' : 'N',
	'core' : r'(.*)(ь)(м)(о)',
	'inflex' : 'ь|е',
	'flex' : 'о	=nom	а	у	ѣ	омъ	=nom	а^	=nom	2ъ	ѡмъ/амъ	ѣхъ/ахъ	ы/ами	ѣ^	у^	ома'
	},
'N2j' : {
	'type' : 'N',
	'core' : r'(.*)()()(е)',
	'inflex' : '',
	'flex' : 'е	=nom	я	ю	и/ѣ	емъ	=nom	я^	=nom	ей	ємъ/ямъ	яхъ/ехъ	и^/ями	и^	ю^	ема'
	},
'N2k' : {
	'type' : 'N',
	'core' : r'(.*)(к)()(о)',
	'inflex' : 'к|ц',
	'flex' : 'о	=nom	а	у	2ѣ/ѣ	омъ	=nom	а^	=nom	ъ	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	2ѣ^	у^	ома'
},
'N2g' : {
	'type' : 'N',
	'core' : r'(.*)(г)()(о)',
	'inflex' : 'г|з',
	'flex' : 'о	=nom	а	у	2ѣ/ѣ	омъ	=nom	а^	=nom	ъ	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	2ѣ^	у^	ома'
	},
'N2h' : {
	'type' : 'N',
	'core' : r'(.*)(х)()(о)',
	'inflex' : 'х|с',
	'flex' : 'о	=nom	а	у	2ѣ/ѣ	омъ	=nom	а^	=nom	ъ	ѡмъ/амъ	2ѣхъ/ахъ	и^/ами	2ѣ^	у^	ома'
	},
'N2s' : {
	'type' : 'N',
	'core' : r'(.*)()(щ|ж|ч)(е)',
	'inflex' : '',
	'flex' : 'е	=nom	а	у	и/ѣ	емъ	=nom	а^	=nom	ъ	ємъ/амъ	ахъ/exъ/ихъ	и^/ами	и^	ю^	ема'
	},
'N2c' : {
	'type' : 'N',
	'core' : r'(.*)()(ц)(е)',
	'inflex' : '',
	'flex' : 'е	=nom	а	у	ѣ/ы	емъ	=nom	а^	=nom	ъ	ємъ/амъ	ахъ/ехъ	ы^/ами	ы^	у^	ема'
	},
'N2c*-a' : {
	'type' : 'N',
	'core' : r'(.*)(ц)()(е)',
	'inflex' : 'ц|ец',
	'flex' : 'е	=nom	а	у	ѣ/ы	емъ	=nom	а^	=nom	2ъ	ємъ/амъ	ахъ/ехъ	ы^/ами	ы^	у^	ема'
	},
'N2c*-b' : {
	'type' : 'N',
	'core' : r'(.*)(ьц)()(е)',
	'inflex' : 'ьц|ец',
	'flex' : 'е	=nom	а	у	ѣ/ы	емъ	=nom	а^	=nom	2ъ	ємъ/амъ	ахъ/ехъ	ы^/ами	ы^	у^	ема'
	},
'N2i' : {
	'type' : 'N',
	'core' : r'(.*)(i)()(е)',
	'inflex' : 'i|',
	'flex' : 'е	=nom	я	ю	и	емъ	=nom	я^	=nom	й	ємъ/ямъ	ихъ/ахъ/яхъ	и^/2ьми/2ми/ями	и^	ю^	ема'
	},
'N2e' : {
	'type' : 'N',
	'core' : r'(.*)(i)()(е)',
	'inflex' : 'i|',
	'flex' : 'е	=nom	а/я	ю	и	емъ	=nom	я^	=nom	й	ємъ/ямъ	ихъ/ахъ/яхъ	и^/2ьми/2ми/ями	и^	ю^	ема'
	},
#_________________________________________________________________________________________________________________
'N3t' : {
	'type' : 'N',
	'core' : r'(.*)()()(а)',
	'inflex' : '',
	'flex' : 'а	у	ы	ѣ	ѣ	ою	о	ы^	=nom/=gen	ъ/ъ^	амъ	ахъ	ами	ѣ^	у^	ама'
	},
'N3t*-a' : {
	'type' : 'N',
	'core' : r'(.*)()(н|р|б)(а)',
	'inflex' : '|е',
	'flex' : 'а	у	ы	ѣ	ѣ	ою	о	ы^	=nom/=gen	2ъ	амъ	ахъ	ами	ѣ^	у^	ама'
	},
'N3t*-b' : {
	'type' : 'N',
	'core' : r'(.*)(ь)(б|м)(а)',
	'inflex' : 'ь|е',
	'flex' : 'а	у	ы	ѣ	ѣ	ою	о	ы^	=nom/=gen	2ъ	амъ	ахъ	ами	ѣ^	у^	ама'
	},
'N3j' : {
	'type' : 'N',
	'core' : r'(.*)()()(я)',
	'inflex' : '',
	'flex' : 'я	ю	и	и/ѣ	и/ѣ	ею	е	и^/я^	=nom/=gen	ь/ей	ямъ	яхъ	ями	и^	ю^	яма'
	},
'N3j*' : {
	'type' : 'N',
	'core' : r'(.*)()(л)(я)',
	'inflex' : '|е',
	'flex' : 'я	ю	и	и/ѣ	и/ѣ	ею	е	и^/я^	=nom/=gen	2ь/ей	ямъ	яхъ	ями	и^	ю^	яма'
	},
'N3k' : {
	'type' : 'N',
	'core' : r'(.*)(к)()(а)',
	'inflex' : 'к|ц',
	'flex' : 'а	у	и	2ѣ/ѣ	2ѣ/ѣ	ою	о	и^	=nom/=gen	ъ	амъ	ахъ	ами	2ѣ^	у^	ама'
	},
'N3g' : {
	'type' : 'N',
	'core' : r'(.*)(г)()(а)',
	'inflex' : 'г|з',
	'flex' : 'а	у	и	2ѣ/ѣ	2ѣ/ѣ	ою	о	и^	=nom/=gen	ъ	амъ	ахъ	ами	2ѣ^	у^	ама'
	},
'N3h' : {
	'type' : 'N',
	'core' : r'(.*)(х)()(а)',
	'inflex' : 'х|с',
	'flex' : 'а	у	и	2ѣ/ѣ	2ѣ/ѣ	ою	о	и^	=nom/=gen	ъ	амъ	ахъ	ами	2ѣ^	у^	ама'
	},
'N3k*-a' : {
	'type' : 'N',
	'core' : r'(.*)(к)()(а)',
	'inflex' : 'к|ок|ц',
	'flex' : 'а	у	и	3ѣ/ѣ	3ѣ/ѣ	ою	о	и^	=nom/=gen	2ъ/ей	амъ	ахъ	ами	3ѣ^	у^	ама'
	},
'N3k*-b' : {
	'type' : 'N',
	'core' : r'(.*)(йк)()(а)',
	'inflex' : 'йк|ек|йц',
	'flex' : 'а	у	и	3ѣ/ѣ	3ѣ/ѣ	ою	о	и^	=nom/=gen	2ъ/ей	амъ	ахъ	ами	3ѣ^	у^	ама'
	},
'N3k*-c' : {
	'type' : 'N',
	'core' : r'(.*)(ьк)()(а)',
	'inflex' : 'ьк|ек|ьц',
	'flex' : 'а	у	и	3ѣ/ѣ	3ѣ/ѣ	ою	о	и^	=nom/=gen	2ъ/ей	амъ	ахъ	ами	3ѣ^	у^	ама'
	},
'N3s' : {
	'type' : 'N',
	'core' : r'(.*)()(ж|ч|ш|щ)(а)',
	'inflex' : '',
	'flex' : 'а	у	и	и/ѣ	и/ѣ	ею	е	и^/ы	=nom/=gen	ъ/ей	амъ	ахъ	ами	и^	у^	ама'
	},
'N3c' : {
	'type' : 'N',
	'core' : r'(.*)()(ц)(а)',
	'inflex' : '',
	'flex' : 'а	у	ы	ѣ	ѣ	ею	е	ы^/а^	=nom/=gen	ъ	амъ	ахъ	ами	ы^	у^	ама'
	},
'N3c*-a' : {
	'type' : 'N',
	'core' : r'(.*)()(ц)(а)',
	'inflex' : '|е',
	'flex' : 'а	у	ы	ѣ	ѣ	ею	е	ы^/а^	=nom/=gen	2ъ	амъ	ахъ	ами	ы^	у^	ама'
	},
'N3c*-b' : {
	'type' : 'N',
	'core' : r'(.*)(ь)(ц)(а)',
	'inflex' : 'ь|е',
	'flex' : 'а	у	ы	ѣ	ѣ	ею	е	ы^/а^	=nom/=gen	2ъ	амъ	ахъ	ами	ы^	у^	ама'
	},
'N3c*-c' : {
	'type' : 'N',
	'core' : r'(.*)(й)(ц)(а)',
	'inflex' : 'й|е',
	'flex' : 'а	у	ы	ѣ	ѣ	ею	е	ы^/а^	=nom/=gen	2ъ	амъ	ахъ	ами	ы^	у^	ама'
	},
'N3a' : {
	'type' : 'N',
	'core' : r'(.*)()()(я)',
	'inflex' : '',
	'flex' : 'я	ю	и	и	и	ею	е	и^/я^	=nom/=gen	й	ямъ	яхъ	ями	и^	ю^	яма'
	},
'N3i' : {
	'type' : 'N',
	'core' : r'(.*)()()(я)',
	'inflex' : '',
	'flex' : 'я	ю	и	и	и	ею	е	и^/я^	=nom/=gen	й	ямъ/амъ	яхъ/ахъ	ями/ами	и^	ю^	яма/ама'
	},
'N3e' : {
	'type' : 'N',
	'core' : r'(.*)()()(а)',
	'inflex' : '',
	'flex' : 'а	ю	и	и	и	ею	е	и^/я^	=nom/=gen	й	ямъ/амъ	яхъ/ахъ	ями/ами	и^	ю^	яма/ама'
	},
#__________________________________________________________________________________________________________________
'N41' : {
	'type' : 'N',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь	=nom	и	и	и	ію	е	и^	=nom/=gen	ей	емъ	ехъ	ьми	и^	ію^	ема/ьма'
	},
'N42' : {
	'type' : 'N',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь	=nom	и	и	и	емъ	ь/и	іе	и^	ій/ей	ємъ	ехъ	ьми	и^	ію	ьма'
	},
'N43' : {
	'type' : 'N',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь	=nom	е	и	и	емъ	=nom	и^	=nom	ій/ей	ємъ	ѣхъ	ьми	и^	ію	ема/ьма'
	},
'N43*' : {
	'type' : 'N',
	'core' : r'(.*)(е)(н)(ь)',
	'inflex' : '|е',
	'flex' : '2ь	=nom	е	и/еви	и	емъ	=nom	іе/и^	=nom	ій/ей	ємъ	ехъ	2ьми	и^	ію/ю^	2ьма'
	},
#_________________________________________________________________________________________________________________
'N5en' : {
	'type' : 'N',
	'core' : r'(.*)(я)()()',
	'inflex' : 'я|ен',
	'flex' : '1	=nom	2е	2и	2и	2емъ	=nom	2а	=nom	2ъ	2ємъ/2ѡмъ	2ѣхъ	2ы^/ми	2и^	2у^	2ема/2ама'
	},
'N5et' : {
	'type' : 'N',
	'core' : r'(.*)(я)()()',
	'inflex' : 'я|ят',
	'flex' : '1	=nom	2е	2и	2и	2емъ	=nom	2а	=nom	2ъ	2ємъ/2ѡмъ	2ѣхъ	2ы	2и^	2у^	2ема/2ама'
	},
'N5at' : {
	'type' : 'N',
	'core' : r'(.*)(а)()()',
	'inflex' : 'а|ат',
	'flex' : '1	=nom	2е	2и	2и	2емъ	=nom	2а	=nom	2ъ	2ємъ/2ѡмъ	2ѣхъ	2ы	2и^	2у^	2ема/2ама'
	},
'N5bes' : {
	'type' : 'N',
	'core' : r'(.*)(о)()()',
	'inflex' : 'о|ес',
	'flex' : '1	=nom	2е	2и	2и	2емъ	=nom	2а	=nom	2ъ	2ємъ	2ѣхъ	2ы	2и^	2у^	2ема'
	},
'N5hes' : {
	'type' : 'N',
	'core' : r'(.*)(хо)()()',
	'inflex' : 'хо|шес',
	'flex' : '1	=nom	2е	2и	2и	2емъ	=nom	2а	=nom	2ъ	2ємъ	2ѣхъ	2ы	2и^	2у^	2ема'
	},
'N5er' : {
	'type' : 'N',
	'core' : r'(.*)(и)()()',
	'inflex' : 'и|ер',
	'flex' : '1	2ь	2е	2и	2и	2ію	=nom	2и^	=nom/=gen	2ій/2ей	2емъ	2ехъ	2ьми	2и^	2ію^	2ема'
	},
'N5ov' : {
	'type' : 'N',
	'core' : r'(.*)(ы)()()',
	'inflex' : 'ы|ов',
	'flex' : '1	2ь	2е	2и	2и	2ію	=nom	2и^	=nom/=gen	2ей	2амъ	2ахъ	2ами	2и^	2ію^	2ама'
	},
'N5*ov' : {
	'type' : 'N',
	'core' : r'(.*)(о)(в)(ь)',
	'inflex' : '|о',
	'flex' : '2ь	=nom	е	и	и	2ію	=nom	2и^	=nom	ей	амъ	ахъ	ами	и^	2ію^	ама'
	},
#_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'A1sh' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ъ|ь|#)',
	'inflex' : '',
	'flex' : 'шій	ъ/ь/шъ/шь	ше/шо	ша	шу	ши	шимъ	ши	шу	ши	ши	шею	ше	шя/шы	ша^	шихъ	шымъ	шими	ша^	шѣ^/шѣ	шую^	шема	шій/ый/ій	шее	шагѡ	шаго	шему	шемъ	шая	шую	шія	шей	шіи	шыя	шая^	шая^	шіи	шую^	шима	-	-	-	шіе'
	},
'A1t' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ый|#)',
	'inflex' : '',
	'flex' : 'ый	ъ/ь	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-a' : {
	'type' : 'A1',
	'core' : r'(.*)()(н)(ый)',
	'inflex' : '|е',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-b' : {
	'type' : 'A1',
	'core' : r'(.*)(ь)(н)(ый)',
	'inflex' : 'ь|е',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-c' : {
	'type' : 'A1',
	'core' : r'(.*)(й)(н)(ый)',
	'inflex' : 'й|е',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-d' : {
	'type' : 'A1',
	'core' : r'(.*)(й)(н)(ый)',
	'inflex' : 'й|и',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-e' : {
	'type' : 'A1',
	'core' : r'(.*)()(л)(ый)',
	'inflex' : '|е',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1t*-e' : {
	'type' : 'A1',
	'core' : r'(.*)()(л)(ый)',
	'inflex' : '|е',
	'flex' : 'ый	2ъ	о	а	у	ѣ	ымъ	а	у	ы	ѣ	ою	и	ы^	а^	ыхъ	ымъ^	ыми	а^	ѣ^	у^	ома	ый	ое	агѡ	аго	ому	ѣмъ/омъ	ая	ую	ыя	ѣй/ой	іи	ыя^	ая^	ая^	ѣи	ую^	ыма	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1n*' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ный|#)',
	'inflex' : '',
	'flex' : 'ный	ъ/енъ	о/но	а/на	у/ну	ѣ/нѣ	нымъ/ымъ	а/на	у/ну	ы/ны	ѣ/нѣ	ою/ною	и/ни	ы^/ны^	а^/на^	ныхъ/ыхъ	нымъ^/ымъ^	ыми/ными	а^/на^	ѣ^/нѣ^	у^/ну^	нома/ома	ный/ый	ное/ое	нагѡ/агѡ	наго/аго	ному/ому	нѣмъ/номъ/ѣмъ/омъ	ная/ая	ную/ую	ныя/ыя	нѣй/ной/ѣй/ой	ніи/іи	ныя^/ыя^	ная^/ая^	ная^/ая^	нѣи/ѣи	ную^/ую^	ныма/ыма	нѣй+$A1sh/ѣй+$A1sh	-	не/е	ніе/іе'
	},
'A1j' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ій)',
	'inflex' : '',
	'flex' : 'ій	ь	е	я	ю	и/ѣ	имъ/ѣмъ	я	ю	и	и/ѣ	ею	и^	и^	я	ихъ	имъ^	ими	я^	и^	ю^	ема	ій	ее	ягѡ	яго	ему	емъ	яя	юю	ія	ей/ѣй	іи	ія^	яя^	яя^	іи	юю^	има	ѣй+$A1sh	ь+$A1sh	е	іе'
	},
'A1j*-a' : {
	'type' : 'A1',
	'core' : r'(.*)()(н)(ій)',
	'inflex' : '|е',
	'flex' : 'ій	2ь/2ъ	е	я	ю	и/ѣ	имъ/ѣмъ	я	ю	и	и/ѣ	ею	и^	и^	я	ихъ	имъ^	ими	я^	и^	ю^	ема	ій	ее	ягѡ	яго	ему	емъ	яя	юю	ія	ей/ѣй	іи	ія^	яя^	яя^	іи	юю^	има	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1j*-b' : {
	'type' : 'A1',
	'core' : r'(.*)(ь)(н)(ій)',
	'inflex' : 'ь|е',
	'flex' : 'ій	2ь/2ъ	е	я	ю	и/ѣ	имъ/ѣмъ	я	ю	и	и/ѣ	ею	и^	и^	я	ихъ	имъ^	ими	я^	и^	ю^	ема	ій	ее	ягѡ	яго	ему	емъ	яя	юю	ія	ей/ѣй	іи	ія^	яя^	яя^	іи	юю^	има	ѣй+$A1sh	$A1sh	е	іе'
	},
'A1k' : {
	'type' : 'A1',
	'core' : r'(.*)(к)()(iй)',
	'inflex' : 'к|ц|ч',
	'flex' : 'ій	ъ	о	а	у	2ѣ	имъ	а	у	и	2ѣ	ою	2ы/2и	и^	а^	ихъ	имъ^	ими	а^	2ѣ^	у^	ома	ій	ое	агѡ	аго	ому	2ѣмъ/омъ	ая	ую	ія	2ѣй/2ей/ой	2ыи/2іи/іи	ія^	ая^	ая^	2ѣи	ую^	има	3ай+$A1sh	-	3е	іе'
	},
'A1sk' : {
	'type' : 'A1',
	'core' : r'(.*)(ск)()(ій)',
	'inflex' : 'ск|ст|#сч',
	'flex' : 'ій	ъ	о	а	у	2ѣ	имъ	а	у	и	2ѣ	ою	2ы/2и	и^	а^	ихъ	имъ^	ими	а^	2ѣ^	у^	ома	ій	ое	агѡ	аго	ому	2ѣмъ/омъ	ая	ую	ія	2ѣй/2ей/ой	2ыи/2іи/іи	ія^	ая^	ая^	2ѣи	ую^	има	3ай+$A1sh	-	3е	іе'
	},
'A1ck' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ій)',
	'inflex' : 'цк|цт|#цч',
	'flex' : 'ій	ъ	о	а	у	2ѣ	имъ	а	у	и	2ѣ	ою	2ы/2и	и^	а^	ихъ	имъ^	ими	а^	2ѣ^	у^	ома	ій	ое	агѡ	аго	ому	2ѣмъ/омъ	ая	ую	ія	2ѣй/2ей/ой	2ыи/2іи/іи	ія^	ая^	ая^	2ѣи	ую^	има	3ай+$A1sh	-	3е	іе'
	},
'A1g' : {
	'type' : 'A1',
	'core' : r'(.*)(г)()(iй)',
	'inflex' : 'г|з|ж',
	'flex' : 'ій	ъ	о	а	у	2ѣ	имъ	а	у	и	2ѣ	ою	2и	и^	а^	ихъ	имъ^	ими	а^	2ѣ^	у^	ома	ій	ое	агѡ	аго	ому	2ѣмъ/омъ	ая	ую	ія	2ѣй/2ей/ой	2іи/іи	ія^	ая^	ая^	2ѣи	ую^	има	3ай+$A1sh	-	3е	іе'
	},
'A1h' : {
	'type' : 'A1',
	'core' : r'(.*)(х)()(ій)',
	'inflex' : 'х|с|ш',
	'flex' : 'ій	ъ	о	а	у	2ѣ	имъ	а	у	и	2ѣ	ою	2и	и^	а^	ихъ	имъ^	ими	а^	2ѣ^	у^	ома	ій	ое	агѡ	аго	ому	2ѣмъ/омъ	ая	ую	ія	2ѣй/2ей/ой	2іи/іи	ія^	ая^	ая^	2ѣи	ую^	има	3ай+$A1sh	-	3е	іе'
	},
'A1k*-a' : {
	'type' : 'A1',
	'core' : r'(.*)(к)()(ій)',
	'inflex' : 'к|ок|ц|ч',
	'flex' : 'ій	2ъ	о	а	у	3ѣ	имъ	а	у	и	3ѣ	ою	3ы	и^	а^	ихъ	имъ^	ими	а^	3ѣ^	у^	ома	ій	ое	агѡ	аго	ому	3ѣмъ/омъ	ая	ую	ія	3ѣй/3ей/ой	3ыи/2іи/іи	ія^	ая^	ая^	3ѣи	ую^	има	4ай+$A1sh	-	4е	іе'
	},
'A1k*-b' : {
	'type' : 'A1',
	'core' : r'(.*)(к)()(ій)',
	'inflex' : 'к|ек|ц|ч',
	'flex' : 'ій	2ъ	о	а	у	3ѣ	имъ	а	у	и	3ѣ	ою	3ы	и^	а^	ихъ	имъ^	ими	а^	3ѣ^	у^	ома	ій	ое	агѡ	аго	ому	3ѣмъ/омъ	ая	ую	ія	3ѣй/3ей/ой	3ыи/2іи/іи	ія^	ая^	ая^	3ѣи	ую^	има	4ай+$A1sh	-	4е	іе'
	},
'A1k*-c' : {
	'type' : 'A1',
	'core' : r'(.*)(ьк)()(ій)',
	'inflex' : 'ьк|ек|ц|ч',
	'flex' : 'ій	2ъ	о	а	у	3ѣ	имъ	а	у	и	3ѣ	ою	3ы	и^	а^	ихъ	имъ^	ими	а^	3ѣ^	у^	ома	ій	ое	агѡ	аго	ому	3ѣмъ/омъ	ая	ую	ія	3ѣй/3ей/ой	3ыи/2іи/іи	ія^	ая^	ая^	3ѣи	ую^	има	4ай+$A1sh	-	4е	іе'
	},
'A1k*-d' : {
	'type' : 'A1',
	'core' : r'(.*)(йк)()(ій)',
	'inflex' : 'йк|ек|йц|йч',
	'flex' : 'ій	2ъ	о	а	у	3ѣ	имъ	а	у	и	3ѣ	ою	3ы	и^	а^	ихъ	имъ^	ими	а^	3ѣ^	у^	ома	ій	ое	агѡ	аго	ому	3ѣмъ/омъ	ая	ую	ія	3ѣй/3ей/ой	3ыи/2іи/іи	ія^	ая^	ая^	3ѣи	ую^	има	4ай+$A1sh	-	4е	іе'
	},
'A1s' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ій)',
	'inflex' : '',
	'flex' : 'ій	ь/ъ	е/о	а	у	и	имъ	а	у	и	и	ею	и^	и/ы	я	ихъ	имъ^/ымъ	ими	а^	и^	у^	ема	ій	ее	агѡ	аго	ему	емъ	ая	ую	ія	ей	іи	ыя	ая^	ая^	іи	ую^	има	ай+$A1sh	-	-	іе'
	},
'A1sx' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ій|#)',
	'inflex' : '',
	'flex' : 'ій	ъ/ь	е/о	а	у	емъ	имъ	и	у	и	и	ею	е	я/ы	а^	ихъ	имъ^/ымъ	ими	а^	ѣ^/ѣ	ую^	ема	ій	ее	агѡ	аго	ему	емъ	ая	ую	ія	ей	іи	ыя	ая^	ая^	іи	ую^	има	-	-	-	іе'
	},
'A1a' : {
	'type' : 'A1',
	'core' : r'(.*)()()(ій)',
	'inflex' : '',
	'flex' : 'ій	й	е	я	ю	и	имъ	я	ю	и	и	ею	и^	и	я	ихъ	имъ^	ими	я^	и^	ю^	ема	ій	ее	ягѡ	яго	ему	емъ	яя	юю	ія	ей	іи	ія^	яя^	яя^	іи	юю^	има	яй+$A1sh	-	е	іе'
	},
'A2i' : {
	'type' : 'A1',
	'core' : r'(.*)()()(й)',
	'inflex' : '',
	'flex' : '-	й	е	я	ю	и	имъ	я	ю	и	и	ею	и^	и	я	ихъ	имъ^	ими	я^	и^	ю^	ема	-	-	ягѡ	яго	ему	емъ	-	-	-	ей	-	-	-	-	-	-	има	-	-	-	-'
	},
#_______________________________________________________________________________________________________________________________________________________________________________________________________________________________
'A2t' : {
	'type' : 'A2',
	'core' : r'(.*)()()(ъ)',
	'inflex' : '',
	'flex' : 'ъ	ъ	о	а	у	ѣ/омъ	ымъ	а	у	ы	ѣ/ой	ою	ы/и	ы^	а^	ыхъ/ѣхъ	ымъ^	ы^/ыми	а^	ѣ^	у^	ыма/ома'
	},
'A2j' : {
	'type' : 'A2',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь	ь	е	я	ю	и/ѣ	имъ	я	ю	и	и/ѣ	ею	и^	и	я^	ихъ	имъ^	и^/ими	я^	и^	ю^	има/ема'
	},
'#A1j*' : {
	'type' : 'A2',
	'core' : r'(.*)()(н)(ій)',
	'inflex' : '|е',
	'flex' : 'ій	2ь	е	я	ю	и	имъ	я	ю	и	и	ею	и^	и	я	ихъ	имъ^	ими	я^	и^	ю^	има'
	},
'#A1s/A1i' : {
	'type' : 'A2',
	'core' : r'(.*)()(ч)(ій)',
	'inflex' : '|е',
	'flex' : '2ь	2ь	е	а	у	и	имъ	а	у	и	и	еу	и^	и	а	ихъ	имъ^	и^/ими	а^	и^	у^	има'
	},
#___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'PA1' : {
	'type' : 'PA1',
	'core' : r'(.*)()()(ъ)',
	'inflex' : '',
	'flex' : 'ъ	о	огѡ	ого	ому	омъ	ѣмъ	а	у	оя	ой/ѣ	ою	и	ы^	а^	ѣхъ	ѣмъ^	ѣми	а^	ѣ^	ѡю	ѣма	ый	ое	агѡ	аго	-	ымъ	ая	ую	ыя	ѣй/ой	іи	ыя/ыя^	ая^	ыхъ	ымъ^	ыми	ая^	ѣи	ѡю	ыма'
	},
'PA1k' : {
	'type' : 'PA1',
	'core' : r'(.*)(к)()(ъ)',
	'inflex' : 'к|ц',
	'flex' : 'ъ	о	огѡ/а	ого/а	ому/у	омъ/2ѣмъ	2ѣмъ	а	у	оя	ой/ѣ	ою	2ы/2и	и	а^	2ѣхъ	2ѣмъ^	2ѣми	а^	2ѣ^	ѡю	2ѣма	ій	ое	агѡ	аго	-	имъ	ая	ую	ія	2ѣй/ой	2ыи/2іи	ія/ія^	ая^	ихъ	имъ^	ими	ая^	2ѣи	ѡю	има'
	},
#___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'PA2t' : {
	'type' : 'PA2',
	'core' : r'(.*)()()(ой)',
	'inflex' : '',
	'flex' : 'ой	ое/о	огѡ	ого	ому	омъ	ѣмъ	ая/а	у/ую	оя	ой	ою	іи/и	ыя/ы^	ая^/а^	ѣхъ	ѣмъ^	ѣми	а^	ѣ^/а^	ѡю	ѣма'
	},
'PA2j' : {
	'type' : 'PA2',
	'core' : r'(.*)()()(ей)',
	'inflex' : '',
	'flex' : 'ей	іе	егѡ	его	ему	емъ	имъ	ія	ію	ея	ей	ею	іи	ія^	ія	ихъ	имъ^	ими	ія^	ія^	ею^	има'
	},
'PA2j*' : {
	'type' : 'PA2',
	'core' : r'(.*)(е)(с)(ь)',
	'inflex' : '|е',
	'flex' : '2ь	е	егѡ	его	ему	емъ	имъ	я	ю	ея	ей	ею	и	я^	я^	ѣхъ	ѣмъ^	ѣми	я^	ѣ^/я^	ею^	ѣма'
	},
'PA2k' : {
	'type' : 'PA2',
	'core' : r'(.*)()()(ій)',
	'inflex' : '',
	'flex' : 'ій	ое	оегѡ	оего	оему	оемъ	іимъ/имъ	ая	ую	оея	оей	оею	іи	ія^	ая^	іихъ/ихъ	іимъ^/имъ^	іими/ими	ая^	іи	оею^	іима/има'
	},
'PA2s' : {
	'type' : 'PA2',
	'core' : r'(.*)()()(ъ)',
	'inflex' : '',
	'flex' : 'ъ	е	егѡ	его	ему	емъ	имъ	а	у	ея	ей	ею	и	я/ы	а	ихъ	имъ^/ымъ	ими	а^	и	ею^	има'
	},
'PA2a' : {
	'type' : 'PA2',
	'core' : r'(.*)()()(й)',
	'inflex' : '',
	'flex' : 'й	е	егѡ	его	ему	емъ	имъ	я	ю	ея	ей	ею	и	я^	я^	ихъ	имъ^	ими	я^	и	ею^	има'
	},
#________________________________________________________________________________________________________________________________________________________________________________________________________________________
'PA2i' : {
	'type' : 'PA2i',
	'core' : r'(.*)()()(и)',
	'inflex' : '',
	'flex' : '-	е/не	егѡ/негѡ	его/и/нь	ему/нему	емъ/немъ	имъ/нимъ	-	ю/ню	ея/нея	ей/ней	ею/нею	-	я/ня/ихъ/нихъ	я/ня	ихъ/нихъ	имъ/нимъ	ими/ними	-	я/ня	ею/нею	има/нима'
	},
'PA2izhe' : {
	'type' : 'PA2i',
	'core' : r'(.*)()()(и)',
	'inflex' : '',
	'flex' : 'и	е/не	егѡ/негѡ	его/и/нь	ему/нему	емъ/немъ	имъ/нимъ	я	ю/ню	ея/нея	ей/ней	ею/нею	е	я/ня/ихъ/нихъ	я/ня	ихъ/нихъ	имъ/нимъ	ими/ними	-	я/ня	ею/нею	има/нима'
	},
#________________________________________________________________________________________________________________________________________________________________________________________________________________________
'PNja' : {
	'type' : 'PN',
	'core' : r'(.*)()()(азъ)',
	'inflex' : '',
	'flex' : 'азъ	менє	мене	мнѣ	мнѣ	мною	мя	ми	-	-	-	-'
	},
'PNty' : {
	'type' : 'PN',
	'core' : r'(.*)()(т)(ы)',
	'inflex' : '',
	'flex' : 'ы	ебє	ебе	ебѣ	ебѣ	обою	я	и	-	-	-	-'
	},
'PNseb' : {
	'type' : 'PN',
	'core' : r'(.*)()(с)(ебе)',
	'inflex' : '',
	'flex' : '-	ебє	ебе	ебѣ	ебѣ	обою	я	и	-	-	-	-'
	},
'PNmy' : {
	'type' : 'PN',
	'core' : r'(.*)()()(мы)',
	'inflex' : '',
	'flex' : 'мы	насъ	насъ	намъ	насъ	нами	ны	-	мы	ны	наю	нама'
	},
'PNvy' : {
	'type' : 'PN',
	'core' : r'(.*)()(в)(ы)',
	'inflex' : '',
	'flex' : 'ы	асъ	асъ	амъ	асъ	ами	ы	-	ы	ы	аю	ама'
	},
'PNkto' : {
	'type' : 'PN',
	'core' : r'(.*)()(к)(то)',
	'inflex' : '',
	'flex' : 'то	огѡ	ого	ому	омъ	имъ	-	-	-	-	-	-'
	},
'PNcto' : {
	'type' : 'PN',
	'core' : r'(.*)()(ч)(то)',
	'inflex' : '',
	'flex' : 'то	егѡ/есѡ/есогѡ/есо	то	ему/есому	емъ/есомъ	имъ'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'NUM2' : {
	'type' : 'NUM2',
	'core' : r'(.*)()(дв)(а)',
	'inflex' : '',
	'flex' : 'а	ѣ	ою/у	ѣма'
	},
'NUMoba' : {
	'type' : 'NUM2',
	'core' : r'(.*)()(об)(а)',
	'inflex' : '',
	'flex' : 'а	ѣ	ою	ѣма'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'NUM3' : {
	'type' : 'NUM3',
	'core' : r'(.*)()(тр)(и)',
	'inflex' : '',
	'flex' : 'и/іе	ехъ	емъ	еми'
	},
'NUM4' : {
	'type' : 'NUM3',
	'core' : r'(.*)()(четыр)(е)',
	'inflex' : '',
	'flex' : 'е/и	ехъ	емъ	ьми'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'#N41' : {
	'type' : '#NUM',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь	и	и	и	ію	-	-	-	-	-	-	-	-'
	},
'N10' : {
	'type' : '#NUM',
	'core' : r'(.*)()()(ь)',
	'inflex' : '',
	'flex' : 'ь|е	и	и	и	ію	и/е	ъ/ихъ	ехъ/ихъ	емъ/имъ	ьми	и/ѣ	y	ьма'
	},
'#NUM100' : {
	'type' : '#NUM',
	'core' : r'(.*)()()(о)',
	'inflex' : '',
	'flex' : 'о	а	у	ѣ	омъ	а^	ъ^	ѣхъ	ѡмъ	ы	ѣ^	y^	ома'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'NUMoe' : {
	'type' : 'NUMoe',
	'core' : r'(.*)()()(е)',
	'inflex' : '',
	'flex' : 'е	ихъ	имъ	ими	и	я'
	},
'NUMero' : {
	'type' : 'NUMoe',
	'core' : r'(.*)()()(ы)',
	'inflex' : '',
	'flex' : 'ы	ыхъ	ымъ	ыми	ы	ы'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'A0' : {
	'type' : 'A0',
	'core' : r'(.*)()()(ъ|#)',
	'inflex' : '',
	'flex' : 'ъ	а	о	и	а^	и'
	},
'A0*' : {
	'type' : 'A0',
	'core' : r'(.*)(е)(л|гл|ргл)(ъ|#)',
	'inflex' : '|е',
	'flex' : '2ъ	а	о	и	а^	и'
	},
'A0**' : {
	'type' : 'A0',
	'core' : r'(.*)(че)(л)(ъ|#)',
	'inflex' : 'оч|че',
	'flex' : '2ъ	а	о	и	а^	и'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'V0' : {
	'type' : 'V0',
	'core' : r'(.*)()()(#)',
	'inflex' : '',
	'flex' : 'а	ѣ'
	},
#____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
'V21n' : {
	'type' : 'V',
	'core' : r'(.*)()()(ити)',
	'inflex' : '',
	'flex' : 'ити	ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/ь+$A1sh/+$A1sh	им+$A1t	ен+$A1n*'
	},
'V21a' : {
	'type' : 'V',
	'core' : r'(.*)()()(ити)',
	'inflex' : '',
	'flex' : 'ити	ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	й	йте	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh	им+$A1t	ен+$A1n*'
	},
'V21s' : {
	'type' : 'V',
	'core' : r'(.*)()()(ити)',
	'inflex' : '',
	'flex' : 'ити	у	иши	итъ	имъ	ите	атъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	а/ай/я/яй	ащ+$A1sx	+$A1sh/ив+$A1sh	им+$A1t	ен+$A1n*'
	},
'V21pl' : {
	'type' : 'V',
	'core' : r'(.*)(п)()(ити)',
	'inflex' : 'п|пл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21bl' : {
	'type' : 'V',
	'core' : r'(.*)(б)()(ити)',
	'inflex' : 'б|бл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21vl' : {
	'type' : 'V',
	'core' : r'(.*)(в)()(ити)',
	'inflex' : 'в|вл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21ml' : {
	'type' : 'V',
	'core' : r'(.*)(м)()(ити)',
	'inflex' : 'м|мл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21tvl' : {
	'type' : 'V',
	'core' : r'(.*)(тв)()(ити)',
	'inflex' : 'тв|щвл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21sl' : {
	'type' : 'V',
	'core' : r'(.*)(сл)()(ити)',
	'inflex' : 'сл|шл',
	'flex' : 'ити	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	2яхов+$V0/яхов+$V0	2яст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21dzhd' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ити)',
	'inflex' : 'д|жд',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21dzh' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ити)',
	'inflex' : 'д|ж',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21tsch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ити)',
	'inflex' : 'т|щ',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21tch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ити)',
	'inflex' : 'т|ч',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21ssh' : {
	'type' : 'V',
	'core' : r'(.*)(с)()(ити)',
	'inflex' : 'с|ш',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21zzh' : {
	'type' : 'V',
	'core' : r'(.*)(з)()(ити)',
	'inflex' : 'з|ж',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21stsch' : {
	'type' : 'V',
	'core' : r'(.*)(ст)()(ити)',
	'inflex' : 'ст|щ',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V21zdzhd' : {
	'type' : 'V',
	'core' : r'(.*)(зд)()(ити)',
	'inflex' : 'зд|жд',
	'flex' : 'ити	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	2ахъ/яхъ	2аше/яше	2ахомъ/яхомъ	2асте/ясте	2аху/яху	2ахов+$V0/яхов+$V0	2аст+$V0/яст+$V0	ихъ	и	ихомъ	исте	иша	ихов+$V0	ист+$V0	ил+$A0	-	я/яй	ящ+$A1sx	ив+$A1sh/2ь+$A1sh/2+$A1sh	им+$A1t	2ен+$A1n*'
	},
'V22n' : {
	'type' : 'V',
	'core' : r'(.*)()()(ѣти)',
	'inflex' : '',
	'flex' : 'ѣти	ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	ен+$A1n*/ѣн+$A1n*'
	},
'V22p' : {
	'type' : 'V',
	'core' : r'(.*)(п)()(ѣти)',
	'inflex' : 'п|пл',
	'flex' : 'ѣти	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	ѣн+$A1n*'
	},
'V22b' : {
	'type' : 'V',
	'core' : r'(.*)(б)()(ѣти)',
	'inflex' : 'б|бл',
	'flex' : 'ѣти	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	ѣн+$A1n*'
	},
'V22m' : {
	'type' : 'V',
	'core' : r'(.*)(м)()(ѣти)',
	'inflex' : 'м|мл',
	'flex' : 'ѣти	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	ѣн+$A1n*'
	},
'V22dzhd' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ѣти)',
	'inflex' : 'д|жд',
	'flex' : 'ѣти	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	2ен+$A1n*/ѣн+$A1n*'
	},
'V22tsch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ѣти)',
	'inflex' : 'т|щ',
	'flex' : 'ѣти	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	2ен+$A1n*/ѣн+$A1n*'
	},
'V22tch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ѣти)',
	'inflex' : 'т|ч',
	'flex' : 'ѣти	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	2ен+$A1n*/ѣн+$A1n*'
	},
'V22ssh' : {
	'type' : 'V',
	'core' : r'(.*)(с)()(ѣти)',
	'inflex' : 'с|ш',
	'flex' : 'ѣти	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	2ен+$A1n*/ѣн+$A1n*'
	},
'V22stsch' : {
	'type' : 'V',
	'core' : r'(.*)(ст)()(ѣти)',
	'inflex' : 'ст|щ',
	'flex' : 'ѣти	2у	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	им+$A1t	2ен+$A1n*/ѣн+$A1n*'
	},
'V22s' : {
	'type' : 'V',
	'core' : r'(.*)()(ш|ж|ч|щ)(ати)',
	'inflex' : '',
	'flex' : 'ати	у	иши	итъ	имъ	ите	атъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	а/ай/я/яй	ащ+$A1sx	ав+$A1sh	им+$A1t	ан+$A1n*'
	},
'V22a' : {
	'type' : 'V',
	'core' : r'(.*)()()(яти)',
	'inflex' : '',
	'flex' : 'яти	ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	й	йте	имъ	ив+$V0	йт+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	яхъ	я	яхомъ	ясте	яша	яхов+$V0	яст+$V0	ял+$A0	-	я/яй	ящ+$A1sx	яв+$A1sh	им+$A1t	ян+$A1n*'
	},
'V11a' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : '',
	'flex' : 'ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	й	йте	имъ/емъ	ив+$V0/ев+$V0	йт+$V0	хъ	ше	хомъ	сте	ху	хов+$V0	ст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	я/яй	ющ+$A1sx	в+$A1sh	ем+$A1t	н+$A1n*'
	},
'V11e' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(ти)',
	'inflex' : 'ѣ|',
	'flex' : 'ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	й	йте	имъ/емъ	ив+$V0/ев+$V0	йт+$V0	2яхъ/яхъ	2яше/яше	2яхомъ/яхомъ	2ясте/ясте	2яху/яху	яхов+$V0/2яхов+$V0	яст+$V0/2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	я/яй	ющ+$A1sx	в+$A1sh	ем+$A1t	н+$A1n*'
	},
'V12ovau' : {
	'type' : 'V',
	'core' : r'(.*)(ова)()(ти)',
	'inflex' : 'ова|у',
	'flex' : 'ти	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2йт+$V0	хъ	ше	хомъ	сте	ху	хов+$V0	ст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2я/2яй	2ющ+$A1sx	1в+$A1sh	2ем+$A1t	1н+$A1n*'
	},
'V12evau' : {
	'type' : 'V',
	'core' : r'(.*)(ева)()(ти)',
	'inflex' : 'ева|у',
	'flex' : 'ти	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2йт+$V0	хъ	ше	хомъ	сте	ху	хов+$V0	ст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2я/2яй	2ющ+$A1sx	1в+$A1sh	2ем+$A1t	1н+$A1n*'
	},
'V12evaju' : {
	'type' : 'V',
	'core' : r'(.*)(ева)()(ти)',
	'inflex' : 'ева|ю',
	'flex' : 'ти	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2йт+$V0	хъ	ше	хомъ	сте	ху	хов+$V0	ст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2я/2яй	2ющ+$A1sx	1в+$A1sh	2ем+$A1t	1н+$A1n*'
	},
'V12n' : {
	'type' : 'V',
	'core' : r'(.*)()(л|н|р)(ати)',
	'inflex' : '',
	'flex' : 'ати	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	ахъ/яхъ	аше/яше	ахомъ/яхомъ	асте/ясте	аху/яху	ахов+$V0/яхов+$V0	аст+$V0/яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	я/яй	ющ+$A1sx	ав+$A1sh	ем+$A1t	ан+$A1n*'
	},
'V12p' : {
	'type' : 'V',
	'core' : r'(.*)(п)()(ати)',
	'inflex' : 'п|пл',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ/2яхъ	аше/2яше	ахомъ/2яхомъ	асте/2ясте	аху/2яху	ахов+$V0/2яхов+$V0	аст+$V0/2яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12b' : {
	'type' : 'V',
	'core' : r'(.*)(б)()(ати)',
	'inflex' : 'б|бл',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ/2яхъ	аше/2яше	ахомъ/2яхомъ	асте/2ясте	аху/2яху	ахов+$V0/2яхов+$V0	аст+$V0/2яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12m' : {
	'type' : 'V',
	'core' : r'(.*)(м)()(ати)',
	'inflex' : 'м|мл',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ/2яхъ	аше/2яше	ахомъ/2яхомъ	асте/2ясте	аху/2яху	ахов+$V0/2яхов+$V0	аст+$V0/2яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12i' : {
	'type' : 'V',
	'core' : r'(.*)(им)()(ати)',
	'inflex' : 'им|емл',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ/2яхъ	аше/2яше	ахомъ/2яхомъ	асте/2ясте	аху/2яху	ахов+$V0/2яхов+$V0	аст+$V0/2яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12y' : {
	'type' : 'V',
	'core' : r'(.*)(ым)()(ати)',
	'inflex' : 'ым|ъемл',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ/2яхъ	аше/2яше	ахомъ/2яхомъ	асте/2ясте	аху/2яху	ахов+$V0/2яхов+$V0	аст+$V0/2яст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12v' : {
	'type' : 'V',
	'core' : r'(.*)(в)()(ати)',
	'inflex' : 'в|',
	'flex' : 'ати	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2йт+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2я/2яй	2ющ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12dzhd' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ати)',
	'inflex' : 'д|жд',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12tsch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ати)',
	'inflex' : 'т|щ',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12tch' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ати)',
	'inflex' : 'т|ч',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12stsch' : {
	'type' : 'V',
	'core' : r'(.*)(ст)()(ати)',
	'inflex' : 'ст|щ',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12ssh' : {
	'type' : 'V',
	'core' : r'(.*)(с)()(ати)',
	'inflex' : 'с|ш',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12zzh' : {
	'type' : 'V',
	'core' : r'(.*)(з)()(ати)',
	'inflex' : 'з|ж',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12kch' : {
	'type' : 'V',
	'core' : r'(.*)(к)()(ати)',
	'inflex' : 'к|ч',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12sksch' : {
	'type' : 'V',
	'core' : r'(.*)(ск)()(ати)',
	'inflex' : 'ск|щ',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12gzh' : {
	'type' : 'V',
	'core' : r'(.*)(г)()(ати)',
	'inflex' : 'г|ж',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12hsh' : {
	'type' : 'V',
	'core' : r'(.*)(х)()(ати)',
	'inflex' : 'х|ш',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12cch' : {
	'type' : 'V',
	'core' : r'(.*)(ц)()(ати)',
	'inflex' : 'ц|ч',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12dizhd' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ати)',
	'inflex' : 'д|ижд',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12zdzhd' : {
	'type' : 'V',
	'core' : r'(.*)(зд)()(ати)',
	'inflex' : 'зд|жд',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2а/2ай/2я/2яй	2ущ+$A1sx	ав+$A1sh	2ем+$A1t	ан+$A1n*'
	},
'V12a' : {
	'type' : 'V',
	'core' : r'(.*)()()(яти)',
	'inflex' : '',
	'flex' : 'яти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	й	йте	имъ	ив+$V0	йт+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	яхъ	я	яхомъ	ясте	яша	яхов+$V0	яст+$V0	ял+$A0	-	я/яй	ющ+$A1sx	яв+$A1sh	ем+$A1t	ян+$A1n*'
	},
'V12x' : {
	'type' : 'V',
	'core' : r'(.*)()()(ати)',
	'inflex' : '',
	'flex' : 'ати	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	ый	ущ+$A1sx	ав+$A1sh	-	ан+$A1n*'
	},
'V12x*-a' : {
	'type' : 'V',
	'core' : r'(.*)()(р)(ати)',
	'inflex' : '|е',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2емъ/2имъ	2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2ый	2ущ+$A1sx	ав+$A1sh	2ом+$A1t	ан+$A1n*'
	},
'V12x*-b' : {
	'type' : 'V',
	'core' : r'(.*)(об)(р)(ати)',
	'inflex' : 'об|бе',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2емъ/2имъ	2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2ый	2ущ+$A1sx	ав+$A1sh	2ом+$A1t	ан+$A1n*'
	},
'V12x*-c' : {
	'type' : 'V',
	'core' : r'(.*)()(в)(ати)',
	'inflex' : '|о',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2емъ/2имъ	2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2ый	2ущ+$A1sx	ав+$A1sh	2ом+$A1t	ан+$A1n*'
	},
'V12x*-d' : {
	'type' : 'V',
	'core' : r'(.*)(оз)(в)(ати)',
	'inflex' : 'оз|зо',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2емъ/2имъ	2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2ый	2ущ+$A1sx	ав+$A1sh	2ом+$A1t	ан+$A1n*'
	},
'V12x*-e' : {
	'type' : 'V',
	'core' : r'(.*)(г)(н)(ати)',
	'inflex' : 'г|же',
	'flex' : 'ати	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2емъ/2имъ	2ев+$V0	2ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	2ый	2ущ+$A1sx	ав+$A1sh	2ом+$A1t	ан+$A1n*'
	},
'V13a' : {
	'type' : 'V',
	'core' : r'(.*)()()(ути)',
	'inflex' : '',
	'flex' : 'ути	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ/имъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ухъ	у/е	ухомъ	усте	уша	ухов+$V0	уст+$V0	ул+$A0	-	ый	ущ+$A1sx	ув+$A1sh	-	овен+$A1n*/ут+$A1t'
	},
'V13t' : {
	'type' : 'V',
	'core' : r'(.*)()()(нути)',
	'inflex' : '',
	'flex' : 'нути	ну	неши	нетъ	немъ	нете	нутъ	нев+$V0	нет+$V0	ни	ните	немъ/нимъ	нев+$V0	нит+$V0	няхъ	няше	няхомъ	нясте	няху	няхов+$V0	няст+$V0	нухъ/охъ	ну/е	нухомъ/охомъ	нусте/осте	нуша/оша	нухов+$V0/охов+$V0	нуст+$V0/ост+$V0	л+$A0/нул+$A0	-	ный	нущ+$A1sx	+$A1sh/нув+$A1sh	-	новен+$A1n*/нут+$A1t'
	},
'V13g' : {
	'type' : 'V',
	'core' : r'(.*)(г)()(нути)',
	'inflex' : 'г|ж',
	'flex' : 'нути	ну	неши	нетъ	немъ	нете	нутъ	нев+$V0	нет+$V0	ни	ните	немъ/нимъ	нев+$V0	нит+$V0	няхъ	няше	няхомъ	нясте	няху	няхов+$V0	няст+$V0	нухъ/охъ	ну/2е	нухомъ/охомъ	нусте/осте	нуша/оша	нухов+$V0/охов+$V0	нуст+$V0/ост+$V0	л+$A0/нул+$A0	-	ный	нущ+$A1sx	+$A1sh/нув+$A1sh	-	нут+$A1t/2ен+$A1n*/нен+$A1n*'
	},
'V13k' : {
	'type' : 'V',
	'core' : r'(.*)(к)()(нути)',
	'inflex' : 'к|ч',
	'flex' : 'нути	ну	неши	нетъ	немъ	нете	нутъ	нев+$V0	нет+$V0	ни	ните	немъ/нимъ	нев+$V0	нит+$V0	няхъ	няше	няхомъ	нясте	няху	няхов+$V0	няст+$V0	нухъ/охъ	ну/2е	нухомъ/охомъ	нусте/осте	нуша/оша	нухов+$V0/охов+$V0	нуст+$V0/ост+$V0	л+$A0/нул+$A0	-	ный	нущ+$A1sx	+$A1sh/нув+$A1sh	-	нут+$A1t/2ен+$A1n*/нен+$A1n*'
	},
'V13h' : {
	'type' : 'V',
	'core' : r'(.*)(х)()(нути)',
	'inflex' : 'х|ш',
	'flex' : 'нути	ну	неши	нетъ	немъ	нете	нутъ	нев+$V0	нет+$V0	ни	ните	немъ/нимъ	нев+$V0	нит+$V0	няхъ	няше	няхомъ	нясте	няху	няхов+$V0	няст+$V0	нухъ/охъ	ну/2е	нухомъ/охомъ	нусте/осте	нуша/оша	нухов+$V0/охов+$V0	нуст+$V0/ост+$V0	л+$A0/нул+$A0	-	ный	нущ+$A1sx	+$A1sh/нув+$A1sh	-	нут+$A1t/2ен+$A1n*/нен+$A1n*'
	},
'V14z' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : '',
	'flex' : 'ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	л+$A0	-	ый/я/яй	ущ+$A1sx	+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14t' : {
	'type' : 'V',
	'core' : r'(.*)()()(сти)',
	'inflex' : 'т|',
	'flex' : '2сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый/я/яй	ущ+$A1sx	+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14d' : {
	'type' : 'V',
	'core' : r'(.*)()()(сти)',
	'inflex' : 'д|',
	'flex' : '2сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый/я/яй	ущ+$A1sx	+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14p' : {
	'type' : 'V',
	'core' : r'(.*)()()(сти)',
	'inflex' : '',
	'flex' : 'сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	л+$A0	-	ый/я/яй	ущ+$A1sx	+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14st' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : 'т|',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый	ущ+$A1sx	+$A1sh	-	-'
	},
'V14t*-a' : {
	'type' : 'V',
	'core' : r'(.*)(е)()(сти)',
	'inflex' : 'т||е|ет',
	'flex' : '3сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	3л+$A0*	3лъ	ый/я/яй	ущ+$A1sx	4+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14t*-b' : {
	'type' : 'V',
	'core' : r'(.*)(че)()(сти)',
	'inflex' : 'очт|оч|че|чет',
	'flex' : '3сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	3л+$A0**	3лъ	ый/я/яй	ущ+$A1sx	4+$A1sh	ом+$A1t	ен+$A1n*'
	},
'V14ed' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(сти)',
	'inflex' : 'яд|ѣд|ѣ',
	'flex' : '3сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	-	-	-	-	-	-	-	2охъ	2е	2охомъ	2осте	2оша	2охов+$V0	2ост+$V0	3л+$A0	-	-	-	2+$A1sh	-	2ен+$A1n*'
	},
'V14et' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(сти)',
	'inflex' : 'ящ|ѣт|ѣ',
	'flex' : '3сти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	-	-	-	-	-	-	-	2охъ	2е	2охомъ	2осте	2оша	2охов+$V0	2ост+$V0	3л+$A0	-	-	-	2+$A1sh	-	2ен+$A1n*'
	},
'V14k' : {
	'type' : 'V',
	'core' : r'(.*)()()(щи)',
	'inflex' : 'к|ч|ц|',
	'flex' : '4щи	у	2еши	2етъ	2емъ	2ете	утъ	2ев+$V0	2ет+$V0	3ы	3ыте	3емъ	3ев+$V0	3ыт+$V0	2ахъ	2аше	2ахомъ	2асте	2аху	2ахов+$V0	2аст+$V0	охъ	2е	охомъ	осте	оша	охов+$V0	ост+$V0	л+$A0	-	ій/2а/2я	ущ+$A1sx	+$A1sh	ом+$A1t	2ен+$A1n*'
	},
'V14g' : {
	'type' : 'V',
	'core' : r'(.*)()()(щи)',
	'inflex' : 'г|ж|з|',
	'flex' : '4щи	у	2еши	2етъ	2емъ	2ете	утъ	2ев+$V0	2ет+$V0	3и	3ите	3емъ	3ев+$V0	3ит+$V0	2ахъ	2аше	2ахомъ	2асте	2аху	2ахов+$V0	2аст+$V0	охъ	2е	охомъ	осте	оша	охов+$V0	ост+$V0	л+$A0	-	ій/2а/2я	ущ+$A1sx	+$A1sh	ом+$A1t	2ен+$A1n*'
	},
'V14g*-a' : {
	'type' : 'V',
	'core' : r'(.*)(е)()(щи)',
	'inflex' : 'г|ж|з|ег|е',
	'flex' : '5щи	у	2еши	2етъ	2емъ	2ете	утъ	2ев+$V0	2ет+$V0	3и	3ите	3емъ	3ев+$V0	3ит+$V0	2ахъ	2аше	2ахомъ	2асте	2аху	2ахов+$V0	2аст+$V0	охъ	2е	охомъ	осте	оша	охов+$V0	ост+$V0	4л+$A0*	4лъ	ій	ущ+$A1sx	4+$A1sh	ом+$A1t	2ен+$A1n*'
	},
'V14g*-b' : {
	'type' : 'V',
	'core' : r'(.*)(ре)()(щи)',
	'inflex' : 'ерг|ерж|ерз|ерг|ре',
	'flex' : '5щи	у	2еши	2етъ	2емъ	2ете	утъ	2ев+$V0	2ет+$V0	3и	3ите	3емъ	3ев+$V0	3ит+$V0	2ахъ	2аше	2ахомъ	2асте	2аху	2ахов+$V0	2аст+$V0	охъ	2е	охомъ	осте	оша	охов+$V0	ост+$V0	4л+$A0*	4лъ	ій	ущ+$A1sx	4+$A1sh	ом+$A1t	2ен+$A1n*'
	},
'V14eg' : {
	'type' : 'V',
	'core' : r'(.*)(е)()(щи)',
	'inflex' : 'яг|яж|яз|ег|еж|е',
	'flex' : '6щи	у	2еши	2етъ	2емъ	2ете	утъ	2ев+$V0	2ет+$V0	3и	3ите	3емъ	3ев+$V0	3ит+$V0	-	-	-	-	-	-	-	4охъ/охъ	5е/2е	4охомъ/охомъ	4осте/осте	4оша/оша	4охов+$V0/охов+$V0	4ост+$V0/ост+$V0	4л+$A0	-	-	-	4+$A1sh	-	-'
	},
'V15re' : {
	'type' : 'V',
	'core' : r'(.*)(ре)()(ти)',
	'inflex' : 'р|ер|ре',
	'flex' : '3ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый/я/яй	ущ+$A1sx	2+$A1sh	ом+$A1t	2т+$A1t/ен+$A1n*'
	},
'V15er' : {
	'type' : 'V',
	'core' : r'(.*)(ер)()(ти)',
	'inflex' : 'р|ер|ер',
	'flex' : '3ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый/я/яй	ущ+$A1sx	2+$A1sh	ом+$A1t	2т+$A1t/ен+$A1n*'
	},
'V15ol' : {
	'type' : 'V',
	'core' : r'(.*)(ла)()(ти)',
	'inflex' : 'ол|ла',
	'flex' : '2ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	2хъ	2	2хомъ	2сте	2ша	2хов+$V0	2ст+$V0	2л+$A0	-	я/яй	ющ+$A1sx	2в+$A1sh	ем+$A1t/им+$A1t	ен+$A1t/2н+$A1n*'
	},
'V15or' : {
	'type' : 'V',
	'core' : r'(.*)(ра)()(ти)',
	'inflex' : 'ор|ра',
	'flex' : '2ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	2хъ	2	2хомъ	2сте	2ша	2хов+$V0	2ст+$V0	2л+$A0	-	я/яй	ющ+$A1sx	2в+$A1sh	ем+$A1t/им+$A1t	ен+$A1t/2н+$A1n*'
	},
'V15el' : {
	'type' : 'V',
	'core' : r'(.*)(ле)()(ти)',
	'inflex' : 'ел|ле',
	'flex' : '2ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	и	ите	емъ	ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	я/яй	ющ+$A1sx	2в+$A1sh	-	ен+$A1t'
	},
'V15i' : {
	'type' : 'V',
	'core' : r'(.*)(и)()(ти)',
	'inflex' : 'и|i',
	'flex' : 'ти	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	ясте	2яху	2яхов+$V0	2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2я/2яй	2ющ+$A1sx	в+$A1sh	2ем+$A1t	т+$A1t/2ен+$A1n*'
	},
'V15y' : {
	'type' : 'V',
	'core' : r'(.*)(ы)()(ти)',
	'inflex' : 'ы|ов',
	'flex' : 'ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	й	йте	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	я/яй	ющ+$A1sx	в+$A1sh	ем+$A1t	т+$A1t/2ен+$A1n*'
	},
'V15u' : {
	'type' : 'V',
	'core' : r'(.*)(у)()(ти)',
	'inflex' : 'у|ув',
	'flex' : 'ти	ю	еши	етъ	емъ	ете	ютъ	ев+$V0	ет+$V0	й	йте	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	я/яй	ющ+$A1sx	в+$A1sh	ем+$A1t	т+$A1t/2ен+$A1n*'
	},
'V15e' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(ти)',
	'inflex' : 'ѣ|о',
	'flex' : 'ти	2ю	2еши	2етъ	2емъ	2ете	2ютъ	2ев+$V0	2ет+$V0	2й	2йте	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2я/2яй	2ющ+$A1sx	в+$A1sh	2ем+$A1t	т+$A1t'
	},
'V15im' : {
	'type' : 'V',
	'core' : r'(.*)(я)()(ти)',
	'inflex' : 'я|им|ем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15ym' : {
	'type' : 'V',
	'core' : r'(.*)(ъя)()(ти)',
	'inflex' : 'ъя|ым|ъем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15ozjm' : {
	'type' : 'V',
	'core' : r'(.*)(зя)()(ти)',
	'inflex' : 'зя|озьм|зем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15onm' : {
	'type' : 'V',
	'core' : r'(.*)(ня)()(ти)',
	'inflex' : 'ня|онм|нем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15jm' : {
	'type' : 'V',
	'core' : r'(.*)(я)()(ти)',
	'inflex' : 'я|йм|ем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15n' : {
	'type' : 'V',
	'core' : r'(.*)(я)()(ти)',
	'inflex' : 'я|н|ен',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15an' : {
	'type' : 'V',
	'core' : r'(.*)(а)()(ти)',
	'inflex' : 'а|н|ен',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15am' : {
	'type' : 'V',
	'core' : r'(.*)(а)()(ти)',
	'inflex' : 'а|м|ем',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15en' : {
	'type' : 'V',
	'core' : r'(.*)(я)()(ти)',
	'inflex' : 'я|ен|ен',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15um' : {
	'type' : 'V',
	'core' : r'(.*)(у)()(ти)',
	'inflex' : 'у|м|ум',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/тъ	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	3+$A1sh/в+$A1sh	-	т+$A1t'
	},
'V15a' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : '',
	'flex' : 'ти	ну	неши	нетъ	немъ	нете	нутъ	нев+$V0	нет+$V0	ни	ните	нимъ/немъ	нив+$V0/нев+$V0	нит+$V0	хъ	ше	хомъ	сте	ху	хов+$V0	ст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	ный	нущ+$A1sx	в+$A1sh	-	т+$A1t'
	},
'V15ieiezh' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(ти)',
	'inflex' : 'ѣ|ѣж',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	т+$A1t'
	},
'V15ieiezhd' : {
	'type' : 'V',
	'core' : r'(.*)(ѣ)()(ти)',
	'inflex' : 'ѣ|ѣжд',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	т+$A1t'
	},
'V15v' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : '|в',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/2е	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	т+$A1t/2ен+$A1n*'
	},
'V15yov' : {
	'type' : 'V',
	'core' : r'(.*)(ы)()(ти)',
	'inflex' : 'ы|ов',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/2е	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	т+$A1t/2ен+$A1n*'
	},
'V15uov' : {
	'type' : 'V',
	'core' : r'(.*)(у)()(ти)',
	'inflex' : 'у|ов',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ	/2е	хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	т+$A1t/2ен+$A1n*'
	},
'V12p+V22p' : {
	'type' : 'V',
	'core' : r'(.*)(п)()(ати)',
	'inflex' : 'п|пл',
	'flex' : 'ати	2ю	иши	итъ	имъ	ите	ятъ	ив+$V0	ит+$V0	и	ите	имъ	ив+$V0	ит+$V0	ахъ	аше	ахомъ	асте	аху	ахов+$V0	аст+$V0	ахъ	а	ахомъ	асте	аша	ахов+$V0	аст+$V0	ал+$A0	-	я/яй	ящ+$A1sx	ав+$A1sh	-	ан+$A1n*'
	},
'V22t+V12t' : {
	'type' : 'V',
	'core' : r'(.*)(т)()(ѣти)',
	'inflex' : 'т|щ',
	'flex' : 'ѣти	2у	2еши	2етъ	2емъ	2ете	ятъ/2утъ	2ев+$V0	2ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	-	2ен+$A1n*'
	},
'V22+V14' : {
	'type' : 'V',
	'core' : r'(.*)()(в)(ѣти)',
	'inflex' : '',
	'flex' : 'ѣти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	я/яй	ящ+$A1sx	ѣв+$A1sh	-	ен+$A1n*'
	},
'Viti' : {
	'type' : 'V',
	'core' : r'(.*)(и)()(ти)',
	'inflex' : 'ид|и|ш|ше|шед',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	4л+$A0*	4лъ	ый/я/яй	ущ+$A1sx	5+$A1sh	-	ен+$A1n*'
	},
'Vnaiti' : {
	'type' : 'V',
	'core' : r'(.*)(й)()(ти)',
	'inflex' : 'йд|й|ш|ше|шед',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	4л+$A0*	4лъ	ый/я/яй	ущ+$A1sx	5+$A1sh	-	ен+$A1n*'
	},
'Vzyti' : {
	'type' : 'V',
	'core' : r'(.*)(зы)()(ти)',
	'inflex' : 'зыд|зы|зош|озше|озшед|зоше',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	6л+$A0*	6лъ	ый/я/яй	ущ+$A1sx	5+$A1sh	-	ен+$A1n*'
	},
'Vizyti' : {
	'type' : 'V',
	'core' : r'(.*)(ы)()(ти)',
	'inflex' : 'ыд|ы|ош|ше|шед|оше',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	6л+$A0*	6лъ	ый/я/яй	ущ+$A1sx	5+$A1sh	-	ен+$A1n*'
	},
'Vizjiti' : {
	'type' : 'V',
	'core' : r'(.*)(ъи)()(ти)',
	'inflex' : 'ъид|ъи|ош|ше|шед|оше',
	'flex' : '2ти	у	еши	етъ	емъ	ете	утъ	ев+$V0	ет+$V0	и	ите	имъ/емъ	ив+$V0/ев+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	6л+$A0*	6лъ	ый/я/яй	ущ+$A1sx	5+$A1sh	-	ен+$A1n*'
	},
'Vexat' : {
	'type' : 'V',
	'core' : r'(.*)(ха)()(ти)',
	'inflex' : 'ха|д',
	'flex' : 'ти	2у	2еши	2етъ	2емъ	2ете	2утъ	2ев+$V0	2ет+$V0	2и	2ите	2имъ/2емъ	2ив+$V0/2ев+$V0	2ит+$V0	2яхъ	2яше	2яхомъ	2ясте	2яху	2яхов+$V0	2яст+$V0	хъ		хомъ	сте	ша	хов+$V0	ст+$V0	л+$A0	-	2ый	2ущ+$A1sx	в+$A1sh	-	-'
	},
'Vdat' : {
	'type' : 'V',
	'core' : r'(.*)()()(ти)',
	'inflex' : '',
	'flex' : 'ти	мъ	си	стъ	мъ/мы	сте	дутъ/дятъ	в+$V0	ст+$V0	ждь/й	дите/йте	димъ	див+$V0	дит+$V0	-	-	-	-	-	-	-	хъ	/де	хомъ	сте	ша/доша	хов+$V0	ст+$V0	л+$A0	-	-	-	в+$A1sh	-	н+$A1n*'
	},
'Vest' : {
	'type' : 'V',
	'core' : r'(.*)()()(сти)',
	'inflex' : 'д|',
	'flex' : '2сти	2мъ	2си	2стъ/2сть	2мы	2сте	ятъ	2в+$V0	2ст+$V0	2ждь	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	охъ	е	охомъ	осте	оша	охов+$V0	ост+$V0	2л+$A0	-	ый	ущ+$A1sx	+$A1sh	ом+$A1t	ен+$A1n*'
	},
'Vved' : {
	'type' : 'V',
	'core' : r'(.*)(д)()(ѣти)',
	'inflex' : 'д|',
	'flex' : 'ѣти	2мъ	2си	2сть	2мы	2сте	ятъ	2в+$V0	2ст+$V0	2ждь	ите	имъ	ив+$V0	ит+$V0	яхъ	яше	яхомъ	ясте	яху	яхов+$V0	яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	ый	ущ+$A1sx/ящ+$A1sx	ѣв+$A1sh	ом+$A1t	ѣн+$A1n*'
	},
'Vima' : {
	'type' : 'V',
	'core' : r'(.*)()()(ѣти|ати)',
	'inflex' : '',
	'flex' : 'ѣти/ати	амъ	аши	атъ/ать	амы	ате	утъ	ав+$V0	ат+$V0	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	ый	ущ+$A1sx	-	-	-'
	},
'Vime' : {
	'type' : 'V',
	'core' : r'(.*)()()(ѣти)',
	'inflex' : '',
	'flex' : 'ѣти	ѣю	ѣеши	ѣетъ	ѣемъ	ѣете	ѣютъ	ев+$V0	ет+$V0	ѣй	ѣйте	ѣимъ	ѣив+$V0	ѣит+$V0	ѣяхъ/яхъ	ѣяше/яше	ѣяхомъ/яхомъ	ѣясте/ясте	ѣяху/яху	ѣяхов+$V0/яхов+$V0	ѣяст+$V0/яст+$V0	ѣхъ	ѣ	ѣхомъ	ѣсте	ѣша	ѣхов+$V0	ѣст+$V0	ѣл+$A0	-	ѣя/ѣяй	ѣющ+$A1sx	ѣв+$A1sh	-	-'
	},
'Vsum' : {
	'type' : 'V',
	'core' : r'(.*)()()(быти)',
	'inflex' : '',
	'flex' : 'быти	есмь	еси	есть	есмы	есте	суть	есв+$V0	ест+$V0	буди	будите	будимъ/будемъ	будив+$V0/будев+$V0	будит+$V0	бяхъ	бяше	бяхомъ	бясте	бяху	бяхов+$V0	бяст+$V0	бѣхъ	бѣ	бѣхомъ	бѣсте	бѣша	бѣхов+$V0	бѣст+$V0	был+$A0	-	сый	сущ+$A1sx	быв+$A1sh	-	-'
	},
'Vero' : {
	'type' : 'V',
	'core' : r'(.*)()()(быти)',
	'inflex' : '',
	'flex' : 'быти	буду	будеши	будетъ	будемъ	будете	будутъ	будев+$V0	будет+$V0	буди	будите	будимъ/будемъ	будив+$V0/будев+$V0	будит+$V0	-	-	-	-	-	-	-	быхъ	бы/бысть	быхомъ	бысте	быша	быхов+$V0	быст+$V0	был+$A0	-	-	-	быв+$A1sh	-	быт+$A1t/бвен+$A1n*'
	},
}

def match_word(word, paradigm):
    if re.match(word, paradigms[paradigm]['core']):
        return True

def generate_forms(word, paradigm):
    #print (word)
    forms = {}
    paradigm_head = paradigm_types[paradigms[paradigm]['type']].split('\t') # list of grammatical forms relevant for variable "paradigm"
    inflices = paradigms[paradigm]['inflex'].split('|') # list of inflexes relevant for variable "paradigm"
    for gramm_form, endings in zip(paradigm_head, paradigms[paradigm]['flex'].split('\t')):
        endings = endings.split('/') # list of flexes possible for a single grammatical form
        variants = len(endings)
        for variant_no, ending in enumerate(endings, 1):
            do_update = True
            if re.match(r'^=.*', ending):
                form = ending
            elif re.match('^-$', ending):
                form = ending
                do_update = False
            elif re.match(r'^[1-9].*', ending) and re.search('\$', ending):
                inflex_no = int(ending[0]) - 1
                ending = ending[1:]
                subparadigm = re.split(r'(\+\$|\$)', ending)
                subword = re.sub(paradigms[paradigm]['core'], r'\1' +
                              inflices[inflex_no] + r'\3' + subparadigm[0], word)
                subforms = generate_forms(subword+"#", subparadigm[2])
                for subgramm, subform in subforms.items():
                    forms.update({gramm_form+'+'+subgramm : subform})
                do_update = False
            elif re.match(r'^[1-9].*', ending):
                inflex_no = int(ending[0]) - 1
                ending = ending[1:]
                form = re.sub(paradigms[paradigm]['core'], r'\1' +
                              inflices[inflex_no] + r'\3' + ending, word)
            elif re.search('\$', ending):
                subparadigm = re.split(r'(\+\$|\$)', ending)
                subword = re.sub(paradigms[paradigm]['core'], r'\1' +
                              inflices[0] + r'\3' + subparadigm[0], word)
                subforms = generate_forms(subword+"#", subparadigm[2])
                #print (subword)
                #print (subparadigm)
                #print (subforms)
                for subgramm, subform in subforms.items():
                    forms.update({gramm_form+'+'+subgramm : subform})
                do_update = False
            else:
                form = re.sub(paradigms[paradigm]['core'], r'\1' +
                              inflices[0] + r'\3' + ending, word)
            if variants > 1:
                gramm = gramm_form + ".%d" % variant_no
            else:
                gramm = gramm_form
            if do_update:
                forms.update({gramm: form})
    return forms

def generate_forms_and_print(word, paradigm):
    forms = generate_forms(word, paradigm)
    for gramm, form in forms.items():
        print('%s %s' % (gramm.ljust(20), form))

# Функции для очищения словоформ от титл и преведения к гражданице

In [3]:
os.chdir("C:\\Users\\Arseny\\Desktop\\repository\\ocs\\dictionary")

rt='с̑'[1]
qt='҃'
def kill_tilda (word):
    word = str(word)
    word = re.sub(r'агг҃ел', r'ангел',word)
    word = re.sub(r'агг҃л', r'ангел',word)
    word = re.sub(r'аг҃гл', r'ангел',word)
    word = re.sub(r'анг҃л', r'ангел',word)
    word = re.sub(r'апс̑л([аоеуЕиыья]|$)', r'апостол\1',word)
    word = re.sub(r'апс̑лс', r'апостольс',word)
    word = re.sub(r'апс̑лми', r'апостолами',word)
    word = re.sub(r'апс̑то?л', r'апостол',word)
    word = re.sub(r'бг҃([аоеуЕиы$])', r'бог\1',word)
    word = re.sub(r'бг҃ъ?$', r'бог',word)
    word = re.sub(r'бг҃([^аоеуЕиы])', r'бого\1',word)
    word = re.sub(r'бго҃', r'бого',word)
    word = re.sub(r'бж҃([аоеуЕиы])', r'бож\1',word)
    word = re.sub(r'бж҃ств', r'божеств',word)
    word = re.sub(r'бж҃тв', r'божеств',word)
    word = re.sub(r'бз҃([еЕи])', r'боз\1',word)
    word =re.sub(r'бл҃з', r'блаз',word)
    word =re.sub(r'бл҃г', r'благ',word)
    word =re.sub(r'блг̑в', r'благов',word)
    word =re.sub(r'блгд̑т', r'благодат',word)
    word =re.sub(r'бл҃ж', r'блаж',word)
    word = re.sub(r'бжс̑тв', r'божеств',word)
    word = re.sub(r'блгс̑в', r'благослов',word)
    word = re.sub(r'блгс̑лов', r'благослов',word)
    word = re.sub(r'блгс̑т', r'благост',word)
    word = re.sub(r'блд̑гт', r'благодат',word)
    word = re.sub(r'ближ̑н', r'ближн',word)
    word = re.sub(r'блс̑гв', r'благослов',word)	
    word = re.sub(r'б(ц|ч)д̑', r'богороди\1',word)
    word =re.sub(r'без̑я', r'безъя',word)
    word =re.sub(r'вл҃к', r'владык',word)
    word =re.sub(r'влд̑чц', r'владычиц',word)
    word =re.sub(r'влд̑(к|ц|ч)', r'влады\1',word)	
    word =re.sub(r'воз̑я', r'возъя',word)
    word =re.sub(r'воскрс̑(н$|ни.)', r'воскресе\1',word)
    word =re.sub(r'воскрс̑л', r'воскресил',word)
    word =re.sub(r'воскрс̑', r'воскрес',word)
    word =re.sub(r'воскс̑рша', r'воскресша',word)
    word =re.sub(r'гл҃гол', r'глагол',word)
    word =re.sub(r'гл҃([аоеуЕиыяю]|$)', r'глагол\1',word)
    word =re.sub(r'гла҃', r'глагола',word)
    word =re.sub(r'гдс̑ьс?тв', r'господств',word)
    word =re.sub(r'гдс̑([^врк])', r'господ\1',word)
    word =re.sub(r'гдс̑в', r'господев',word)
    word =re.sub(r'гдс̑к', r'господск',word)
    word =re.sub(r'гдс̑рн', r'государын',word)
    word =re.sub(r'гдс̑р', r'государ',word)
    word =re.sub(r'глв̑а', r'глава',word)
    word =re.sub(r'глс̑', r'глаc',word)
    word =re.sub(r'гпс̑ж', r'госпож',word)
    word =re.sub(r'дх҃н', r'дохн',word)
    word =re.sub(r'двд̑', r'давид',word)
    word =re.sub(r'дв҃д', r'давид',word)
    word =re.sub(r'дв҃', r'дев',word)
    word =re.sub(r'д҃в', r'дев',word)
    word =re.sub(r'дн҃ь$', r'день',word)
    word =re.sub(r'дс҃([Еи])', r'дус\1',word)
    word =re.sub(r'дх҃([аоуеЕиы]|$)', r'дух\1',word)
    word =re.sub(r'дш҃([аоуеЕиыю]|$)', r'душ\1',word)
    word =re.sub(r'дав̑ше', r'давше',word)
    word =re.sub(r'двж̑а', r'дважды',word)
    word =re.sub(r'двс̑т', r'девст',word)
    word =re.sub(r'дерз̑новенно', r'дерзновенно',word)
    word =re.sub(r'домов̑ная', r'домовная',word)
    word =re.sub(r'ен̑н', r'енн',word)
    word =re.sub(r'евг̑а?л', r'евангел',word)
    word = re.sub(r'епкс̑па', r'епископа',word)
    word = re.sub(r'епс̑ко?п', r'епископ',word)
    word =re.sub(r'ии҃л(т|с)', r'израиль\1',word)
    word =re.sub(r'ии҃л', r'израил',word)
    word =re.sub(r'ии҃с(?!ус)', r'иисус',word)
    word =re.sub(r'(?<!и)ис҃([аоуеЕы]|$)(?!ус)', r'иисус\1',word)
    word = re.sub(r'иер?с̑л', r'иерусал', word)
    word = re.sub(r'избав̑л', r'избавл', word)
    word = re.sub(r'из̑(?!я)', r'из', word)
    word = re.sub(r'из̑я', r'изъя', word)
    word = re.sub(r'кн҃г', r'княг', word)
    word = re.sub(r'кн҃з', r'княз', word)
    word = re.sub(r'кн҃ж', r'княж', word)
    word = re.sub(r'кр҃с', r'крес', word)
    word = re.sub(r'крс̑т', r'крест', word)
    word = re.sub(r'кр҃щ', r'крещ', word)
    word = re.sub(r'кр҃ш', r'креш', word)
    word = re.sub(r'л̑ст', r'лест', word)
    word = re.sub(r'л̑щ', r'льщ', word)	
    word =re.sub(r'мт҃ер', r'матер',word)
    word =re.sub(r'мт҃р', r'матер',word)
    word =re.sub(r'мт҃', r'мат',word)
    word =re.sub(r'мл҃тв', r'молитв',word)
    word =re.sub(r'мл҃итв', r'молитв',word)
    word =re.sub(r'мл҃ст', r'милост',word)
    word =re.sub(r'млс̑т', r'милост',word)
    word =re.sub(r'млс̑рд', r'милосерд',word)
    word =re.sub(r'мл҃ч', r'молч',word)
    word =re.sub(r'мр҃и', r'мари',word)
    word =re.sub(r'мч҃ни', r'мучени',word)
    word =re.sub(r'мч҃ени', r'мучени',word)
    word =re.sub(r'мч҃н([^и])', r'мучени\1',word)
    word =re.sub(r'мд̑р', r'мудр',word)
    word =re.sub(r'мрд̑с', r'мудрос',word)
    word =re.sub(r'мрд̑', r'мудр',word)
    word =re.sub(r'млд̑н', r'младен',word)
    word =re.sub(r'млнд̑', r'младен',word)
    word =re.sub(r'млнд̑', r'младен',word)
    word =re.sub(r'мнс̑т', r'монаст',word)
    word =re.sub(r'мс̑ц', r'месяц',word)
    word =re.sub(r'мс̑ч', r'месяч',word)
    word =re.sub(r'м(ц|ч)с̑', r'меся\1',word)
    word =re.sub(r'нн҃Е', r'нынЕ',word)
    word =re.sub(r'нб҃([аоеуЕ])', r'неб\1',word)
    word =re.sub(r'нб҃с', r'небес',word)
    word =re.sub(r'н҃бс', r'небес',word)
    word =re.sub(r'нбс̑', r'небес',word)
    word =re.sub(r'ндл̑', r'недел',word)
    word =re.sub(r'нлд̑', r'недел',word)
    word = re.sub(r'от҃ч', r'отеч',word)
    word = re.sub(r'отч҃', r'отече',word)
    word = re.sub(r'оч҃ь', r'отечь',word)
    word = re.sub(r'оч҃е?(?=с[кт])', r'отече',word)
    word = re.sub(r'оч҃', r'отч',word)
    word =re.sub(r'оц҃$', r'отец',word)
    word =re.sub(r'оц҃', r'отц',word)
    word =re.sub(r'пл҃т', r'плот',word)
    word =re.sub(r'пл҃щ', r'площ',word)
    word =re.sub(r'прпд̑бн$', r'преподобен',word)
    word =re.sub(r'прпд҃б', r'преподоб',word)
    word =re.sub(r'прпд̑б', r'преподоб',word)
    word =re.sub(r'прпд̑н', r'преподобн',word)
    word =re.sub(r'прпд̑бс', r'преподобс',word)
    word =re.sub(r'прс̑н', r'присн',word)
    word =re.sub(r'прс̑тол', r'престол',word)
    word =re.sub(r'псл҃ом', r'псалом',word)
    word =re.sub(r'првд̑н$', r'праведен',word)
    word =re.sub(r'првд̑н(.)', r'праведн\1',word)
    word =re.sub(r'првд̑вн', r'праведн',word)
    word =re.sub(r'првд̑ен', r'праведен',word)
    word =re.sub(r'прд̑вен', r'праведен',word)
    word =re.sub(r'прд̑е', r'пред',word)	
    word =re.sub(r'прд̑те?ч', r'предтеч',word)
    word =re.sub(r'пнд̑е', r'понедельник',word)
    word =re.sub(r'пнд̑л', r'понедел',word)
    word =re.sub(r'под̑я', r'подъя',word)
    word =re.sub(r'про̑рк', r'пророк',word)
    word =re.sub(r'прро̑к', r'пророк',word)
    word =re.sub(r'прро̑чц', r'пророчиц',word)
    word =re.sub(r'ржд̑н', r'рожден',word)
    word =re.sub(r'ржс̑т', r'рождест',word)
    word = re.sub(r'см҃рт', r'смерт',word)
    word = re.sub(r'смр̑т', r'смерт',word)
    word =re.sub(r'сп҃с', r'спас',word)
    word =re.sub(r'спс̑н', r'спасен',word)
    word =re.sub(r'спс̑те', r'спасите',word)
    word =re.sub(r'спс̑', r'спас',word)
    word =re.sub(r'сн҃([аоеуЕиыю]|$)', r'сын\1',word)
    word =re.sub(r'сл҃в', r'слав',word)
    word =re.sub(r'слв̑', r'слов',word)
    word =re.sub(r'сл҃нц', r'солнц',word)
    word =re.sub(r'сл҃не?ч', r'солнеч',word)
    word =re.sub(r'ср҃д', r'серд',word)
    word =re.sub(r'срд̑ц(.)', r'сердц\1',word)
    word =re.sub(r'срд̑ц$', r'сердец',word)
    word =re.sub(r'срд̑ч', r'сердеч',word)
    word =re.sub(r'срцд̑ы', r'сердцы',word)
    word =re.sub(r'сщ҃([аоеуяыи])', r'свящ\1',word)
    word =re.sub(r'ст҃([аоеуяЕыи]|$)', r'свят\1',word)
    word =re.sub(r'прс̑т([аоеуяЕыи]|$)', r'свят\1',word)
    word =re.sub(r'ст҃л', r'святител',word)
    word =re.sub(r'ст҃т', r'святит',word)
    word =re.sub(r'стрс̑т(?!ерп)', r'страст',word)
    word =re.sub(r'стрс̑терп', r'страстотерп',word)
    word = re.sub(r'стх̑р', r'стихир', word)
    word =re.sub(r'телст̑в', r'тельств',word)
    word =re.sub(r'тел̑ств', r'тельств',word)
    word =re.sub(r'тел̑н', r'тельн',word)
    word =re.sub(r'тови҃', r'товит',word)
    word =re.sub(r'тро̑(ц|ч)', r'трои\1',word)
    word =re.sub(r'уч҃ни', r'учени',word)
    word =re.sub(r'учн҃и', r'учени',word)
    word =re.sub(r'учн҃(к|ц|ч)', r'учени\1',word)
    word =re.sub(r'уч҃н(к|ц|ч)', r'учени\1',word)
    word =re.sub(r'уч҃те?л', r'учител',word)
    word =re.sub(r'учте?҃л', r'учител',word)
    word =re.sub(r'х҃с', r'христос',word)
    word =re.sub(r'хрс̑т', r'христ',word)
    word =re.sub(r'цр҃([аоеуЕиыюсья])', r'цар\1',word)
    word =re.sub(r'цр҃ц', r'цариц',word)
    word =re.sub(r'цр҃кв(?!ь)', r'церкв',word)
    word =re.sub(r'цр҃квь', r'церковь',word)
    word =re.sub(r'цр҃ков', r'церков',word)
    word =re.sub(r'црс̑(т|к)', r'царст\1',word)
    word =re.sub(r'цс̑рт', r'царст',word)
    word =re.sub(r'ч҃лв', r'челов',word)
    word =re.sub(r'чл҃о?в', r'челов',word)
    word =re.sub(r'чл҃(к|ч|ц)', r'челове\1',word)
    word =re.sub(r'члчс̑к', r'человеческ',word)
    word =re.sub(r'благочс̑т', r'благочест',word)
    word =re.sub(r'благочтс̑и', r'благочести',word)
    word =re.sub(r'чс̑т([^н]|$)', r'чист\1',word)
    word =re.sub(r'чтс̑([^н]|$)', r'чист\1',word)
    word =re.sub(r'чтс̑е?н$', r'честен',word)
    word =re.sub(r'чс̑те?н$', r'честен',word)
    word =re.sub(r'чтс̑н(.)', r'честн\1',word)	
    word =re.sub(r'чс̑тн(.)', r'честн\1',word)
    word =re.sub(r'чтс̑н(.)', r'честн\1',word)	
    word =re.sub(r'единочс̑тн', r'единочестн',word)
    word =re.sub(r'многочс̑тн', r'многочестн',word)
    word =re.sub(r'нечс̑т([^н])', r'нечист\1',word)
    word =re.sub(r'^очтс̑и', r'очисти',word)
    word =re.sub(r'^очс̑ти', r'очисти',word)
    word =re.sub(r'и(м҃ре|мр̑)къ', r'имярек',word)
    word =re.sub(r'нш҃ихъ', r'наших',word)
    word =re.sub(rt, r'',word)
    word =re.sub(qt, r'',word)
    word =re.sub('҂', r'',word)
    word =re.sub(r'ъ$', r'',word)
    word =re.sub(r'Е', r'е',word)
    return word


def kill_orthography(w):
    w = re.sub(r'ы́'[1],r'',w)
    w = re.sub(r'і',r'и',w)
    w = re.sub(r'ѡ',r'о',w)
    w = re.sub(r'ѻ',r'о',w)
    w = re.sub(r'[ѣє]',r'е',w)
    w = re.sub(r'(е|а|о)ѵ',r'\1в',w)
    w = re.sub(r'ѵ',r'и',w)
    w = re.sub(r'ѳ',r'ф',w)
    w = re.sub(r'ѱ',r'пс',w)
    w = re.sub(r'ѯ',r'кс',w)
    w = re.sub(r'ѕ',r'з',w)
    w =re.sub(r'ъ$', r'',w)
    w =re.sub(r'(?<=а)гг(?=(ай|е|ее|еева|еево|ей|ею|ея|ие|ий||я|а|ю||л|ла|лу|ле|лом|лов|лы|ли|льстии|лино)$)', r'ГГ',w)
    w =re.sub(r'гг', r'нг',w)
    w =re.sub(r'гк', r'нк',w)
    w =re.sub(r'гх', r'нх',w)
    w =re.sub(r'ГГ', r'гг',w)
    w =re.sub(r'ъ?\^', r'',w)
    return w

В переменной polyakov храним исходный словарь НКРЯ:

In [4]:
polyakov = pd.read_csv("polyakov_dic.tsv", sep = '\t', converters = {
        'sort': kill_tilda,
        'lex': kill_orthography
    })
polyakov[polyakov['sort']=='мр']

,id,sort,word,lex,pos,flex_type,gram,pp,freq,dup
57788,57789,мр,мр̑##,марк,"S,m,anim,persn",N1k,++,++,223,0


## Словарь Полякова, отсортированный по частотности:

In [5]:
summa = polyakov['freq'].sum()
polyakov = polyakov.assign(permil=lambda x: x.freq/summa*1000000)
polyakov.sort_values(by = 'permil', ascending = False)

,id,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
39157,39158,и,и,и,CONJ,NaN,NaN,NaN,324762,1,85288.214930
24585,24586,в,въ,в,PR,NaN,NaN,NaN,68629,1,18023.182831
58805,58806,на,на,на,PR,NaN,NaN,NaN,55918,1,14685.050599
33908,33909,еси,єси́,быти,"V,ipf,intr",Vbyt,"indic,praes,sg,2p",NaN,53293,1,13995.679415
80516,80517,от,ѿ,от,PR,NaN,NaN,NaN,49396,1,12972.258653
...,...,...,...,...,...,...,...,...,...,...,...
142013,142014,хиосскаго,хі́осскагѡ,хиосский,A,A1k,"plen,sg,m/n,gen",NaN,0,0,0.000000
23241,23242,вседневная,вседне́вная,вседневный,A,A1t*,"plen,sg,f,nom",NaN,0,0,0.000000
61910,61911,начинаемыя,начина́ємыя,начинати,"V,ipf,tran",V11a,"partcp,praes,pass,plen,pl,m,acc|partcp,praes,p...",NaN,0,1,0.000000
66649,66650,непроходно,непрохо́днѡ,непроходно,ADV,NaN,NaN,NaN,0,1,0.000000


## Пишем новый словарь на основе нашего цсл. корпуса:

In [6]:
with open('corpus_simple.txt','r',encoding='utf8') as text:
    text = text.read()
    words = text.split()
dictionary = {}

for word in words:
    if word not in dictionary:
        dictionary[word]=1000000/len(words)
    else:
        dictionary[word] += 1000000/len(words)
with open ('new_dict.txt','w',encoding='utf8') as file:
    file.write('sort'+'\t'+'permil'+'\n')
    for w in dictionary:
        file.write(w+'\t'+str(dictionary[w])+'\n')

In [7]:
our = pd.read_csv('new_dict.txt', sep = '\t')
our.sort_values(by = 'permil', ascending = False)

,sort,permil
11,и,72163.202501
45,еси,15646.999253
71,в,15055.600346
1,на,12107.700476
7,от,11164.559327
...,...,...
68672,светосияньми,0.245802
68674,веселяя,0.245802
68676,плотолюбие,0.245802
68677,ангелми,0.245802


## Добавляем к словарю Полякова местоимения и числительные:
Получается длинный словарь Полякова (переменная `long`).

In [8]:
pronouns = pd.read_excel('addit_dict.xls', sheet_name = 'pronum', converters = {
        'lex': kill_orthography,
        'sort': kill_tilda
})

In [9]:
pronouns = pronouns.assign(permil=lambda x: x.freq/summa*1000000)
pronouns[pronouns['sort']=='я']

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
1999,я,я,и,SPRO,PA2i,"sg,f,nom|pl,m,acc|pl,f,nom/acc|pl,n,acc|du,acc",NaN,1473,1.0,386.835715
2000,я,я́,и,SPRO,PA2i,"sg,f,nom|pl,m,acc|pl,f,nom/acc|pl,n,acc|du,acc",NaN,1930,1.0,506.851956


In [10]:
numbers = pd.read_excel('addit_dict.xls', sheet_name = 'num', converters = {
        'lex': kill_orthography,
        'sort': kill_tilda
})
numbers = numbers.assign(permil=lambda x: x.freq/summa*1000000)
numbers[numbers['sort']=='а']


,sort,word,lex,pos,flex_type,gram,freq,permil
0,а,1000,҂а҃,NUM,NaN,NaN,2,0.525235
40,а,1,а҃,NUM,NaN,NaN,8348,2192.331671


In [11]:
long = pd.concat([polyakov, numbers, pronouns], sort = False)
del long['id']
long.sort_values(by='freq', ascending = False)

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
39157,и,и,и,CONJ,NaN,NaN,NaN,324762,1.0,85288.214930
24585,в,въ,в,PR,NaN,NaN,NaN,68629,1.0,18023.182831
58805,на,на,на,PR,NaN,NaN,NaN,55918,1.0,14685.050599
33908,еси,єси́,быти,"V,ipf,intr",Vbyt,"indic,praes,sg,2p",NaN,53293,1.0,13995.679415
80516,от,ѿ,от,PR,NaN,NaN,NaN,49396,1.0,12972.258653
...,...,...,...,...,...,...,...,...,...,...
112290,руси,руси++,русь,"S,f,inan",N41,"sg,gen/dat/loc",NaN,0,1.0,0.000000
26014,глв,глв̑,глв̑++,NaN,NaN,NaN,NaN,0,0.0,0.000000
67,всея,всея++,весь,APRO,PA2j*,"brev,sg,f,gen",NaN,0,1.0,0.000000
109945,ракс,ра́ка!!++++,рака,NaN,NaN,NaN,NaN,0,0.0,0.000000


In [13]:
long = long.assign(permil = lambda x: x.freq*100000/448301)
long[long['sort']=='всея']

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
24272,всея,всѣ́я,всеяти,"V,pf,tran",V12a,"indic,aor,sg,2p/3p|partcp,fut,act,brev,sg,m/n",NaN,13,0.0,2.899837
66,всея,всея́,весь,APRO,PA2j*,"brev,sg,f,gen",NaN,931,1.0,207.672970
67,всея,всея++,весь,APRO,PA2j*,"brev,sg,f,gen",NaN,0,1.0,0.000000


## Какие слова есть в нашем корпусе, но отсутствуют в НКРЯ?
Записываем их в переменную `delta`.

In [13]:
delta_words = list(set(our['sort'])-set(long['sort']))
delta_freqs = [dictionary[w] for w in delta_words]

In [14]:
delta_df = pd.DataFrame({'sort':delta_words, 
                         'permil':delta_freqs}
                       ).assign(freq = lambda x: (x.permil*406832/100000))

delta_df.to_csv('delta.csv', sep = '\t')

delta_df.sort_values(by='permil',ascending=False)

,sort,permil,freq
8245,чтущим,151.168050,615.0
2009,поющим,127.079482,517.0
4159,притекающим,109.381760,445.0
13206,ектения,62.925237,256.0
13603,воспевающим,59.729815,243.0
...,...,...,...
7315,предпросиявый,0.245802,1.0
7316,открываемый,0.245802,1.0
7318,ресаво,0.245802,1.0
7319,телесох,0.245802,1.0


## А какие из них содержат орфограмму жи\ши\щи?
Записываем их в переменную `zhi_shi`, меняя орфограмму на жы\шы\щы. В датафрейме `zhi_shi_df` те Поляковские слова, которые содержат эту орфограмму. Теперь дублируем строки из датафрейма `long`, меняя в дубликатах орфограмму словоформы на жи\ши\щи. Таким образом, в словаре Полякова теперь можно будет найти ещё 970 слов, которые были и раньше в Полякове, но не находились из-за разницы в орфографии.

In [15]:
zhi_shi = []
for w in list(delta_df['sort']):
    if re.search(r'(ш|щ|ж)и', w):
        zhi_shi.append(re.sub(r'(ш|щ|ж)и', r'\1ы', w))
print(len(zhi_shi))

2804


In [16]:
zhi_shi_df = long[long['sort'].isin(zhi_shi)]
#zhi_shi_df.to_csv('zhi_shi.csv', sep = '\t')
#def i_y_converter(w):
#    w = re.sub(r'(ч|ш|щ|ж)и', r'\1ы',w)
#    return w
zhi_shi_df['sort'] = zhi_shi_df['sort'].apply(lambda w: re.sub(r'(ш|щ|ж)ы', r'\1и',w))
zhi_shi_df

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
1146,алчущим,а́лчущымъ,алкати,"V,ipf,intr",V12t,"partcp,praes,act,plen/brev,pl,dat",NaN,37,0.0,8.253383
3163,богобоящияся,бг҃обоя́щыяся,богобоятися++,"V,ipf,intr,med",V22a,"partcp,praes,act,plen,pl,m,acc|partcp,praes,ac...",NaN,8,0.0,1.784515
4238,богословившия,бг҃ослови́вшыя,богословити,"V,ipf,intr",V21p,"partcp,praet,act,plen,pl,m,acc|partcp,praet,ac...",NaN,1,1.0,0.223064
4239,богословившия,бг҃осло́вившыя,богословити,"V,ipf,intr",V21p,"partcp,praet,act,plen,pl,m,acc|partcp,praet,ac...",NaN,12,1.0,2.676773
4295,богословствующим,бг҃осло́вствующымъ,богословствовати,"V,ipf,intr",V12ov,"partcp,praes,act,plen/brev,pl,dat",NaN,1,0.0,0.223064
...,...,...,...,...,...,...,...,...,...,...
145846,шествовавшия,ше́ствовавшыя,шествовати,"V,ipf,intr",V12ov,"partcp,praet,act,plen,pl,m,acc|partcp,praet,ac...",NaN,1,0.0,0.223064
145874,шествующим,ше́ствующымъ,шествовати,"V,ipf,intr",V12ov,"partcp,praes,act,plen/brev,pl,dat",NaN,6,0.0,1.338386
146586,являющим,явля́ющымъ,являти,"V,ipf,tran",V11a,"partcp,praes,act,plen/brev,pl,dat",NaN,6,0.0,1.338386
146709,ядущим,яду́щымъ,ясти,"V,ipf,tran",Vest,"partcp,praes,act,plen/brev,pl,dat",NaN,18,0.0,4.015159


In [17]:
longer = pd.concat([long, zhi_shi_df], sort = False)
longer[longer['sort']=='мр']

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
57788,мр,мр̑##,марк,"S,m,anim,persn",N1k,++,++,223,0.0,49.743364


## Аналогично поступаем с орфограммой *ия/ia* (наличие и отсутствие второго южнославянского влияния).

In [18]:
delta2df = delta_df[~delta_df['sort'].isin(zhi_shi_df['sort'])]#вычитаем из дельты те, у которых есть жи-ши-чи-щи
delta2df.sort_values(by='permil',ascending=False) #теперь проблема с ектенией: в нашем корпусе -ия-, а в нкря - -иа-

,sort,permil,freq
13206,ектения,62.925237,256.0
471,илия,50.389350,205.0
278,кафизме,43.506902,177.0
11984,отиде,40.311480,164.0
8546,воскресние,31.708420,129.0
...,...,...,...
7241,егр,0.245802,1.0
7243,твояк,0.245802,1.0
7245,сбытая,0.245802,1.0
7246,согрешенни,0.245802,1.0


In [19]:
ia = []
for w in list(delta2df['sort']):
    if re.search('ия', w):
        ia.append (re.sub(r'ия', r'иа',w)) #в массиве iα лежат возможные нкряшные соответствия для дельты2

ia_df = long[long['sort'].isin(ia)] #вот настоящие нкряшные соответствия для дельты2

ia_df['sort'] = ia_df['sort'].apply(lambda w: re.sub(r'иа', r'ия',w)) #то же самое, но меняем а на я

longer = pd.concat([longer, ia_df], sort = False) #присоединяем к лонегру. Теперь в нем 150,622 слов
delta3df = delta2df[~delta2df['sort'].isin(ia_df['sort'])] #вычитаем из дельты2 иашки
delta3df.sort_values(by = 'permil', ascending = False) 


C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sort,permil,freq
278,кафизме,43.506902,177.0
11984,отиде,40.311480,164.0
8546,воскресние,31.708420,129.0
2578,еванглие,29.496205,120.0
4595,раз,27.529791,112.0
...,...,...,...
7235,детельных,0.245802,1.0
7236,богородицыданиил,0.245802,1.0
7237,материими,0.245802,1.0
7238,родитъ,0.245802,1.0


## Орфограмма *зм (см)* в греческих заимствованиях

In [20]:
zm = []
for w in list(delta3df['sort']):
    if re.search('зм', w):
        zm.append (re.sub(r'зм', r'см',w))
zm_df = longer[longer['sort'].isin(zm)]
zm_df['sort'] = zm_df['sort'].apply(lambda w: re.sub(r'см', r'зм',w)) #то же самое, но меняем з на с
zm_df = zm_df[:-2]

longer_with_zhy_ia_zm = pd.concat([longer, zm_df], sort = False)

delta4df = delta3df[~delta3df['sort'].isin(zm_df['sort'])] #вычитаем из дельты3 зм
delta4df.sort_values(by = 'permil', ascending = False)

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sort,permil,freq
11984,отиде,40.311480,164.0
8546,воскресние,31.708420,129.0
2578,еванглие,29.496205,120.0
4595,раз,27.529791,112.0
6327,аллилуйя,27.283989,111.0
...,...,...,...
7233,явлающи,0.245802,1.0
7235,детельных,0.245802,1.0
7236,богородицыданиил,0.245802,1.0
7237,материими,0.245802,1.0


## Орфограмма *и(ы)* после финально-консонантного префикса

In [21]:
ot = []
for w in list(delta4df['sort']):
    if re.search(r'^(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)ъ?и', w):
        ot.append(re.sub(r'^(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)ъ?и',r'\1ы', w))

ot_df = longer_with_zhy_ia_zm[longer_with_zhy_ia_zm['sort'].isin(ot)]
ot1_df = longer_with_zhy_ia_zm[longer_with_zhy_ia_zm['sort'].isin(ot)]

ot_df['sort'] = ot_df['sort'].apply(lambda w: re.sub(r'^(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)ы',r'\1и', w)) #то же самое, но меняем ы на и

ot1_df['sort'] = ot1_df['sort'].apply(lambda w: re.sub(r'^(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)ы',r'\1ъи', w)) #то же самое, но меняем ы на ъи


longer_with_zhy_ia_zm_ot = pd.concat([longer_with_zhy_ia_zm, ot_df, ot1_df], sort = False)

delta5df = delta4df[~delta4df['sort'].isin(ot_df['sort'])]
delta5df = delta5df[~delta5df['sort'].isin(ot1_df['sort'])] #вычитаем из дельты4 от/отъ
delta5df.sort_values(by = 'permil',ascending = False)

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,sort,permil,freq
8546,воскресние,31.708420,129.0
2578,еванглие,29.496205,120.0
4595,раз,27.529791,112.0
6327,аллилуйя,27.283989,111.0
5652,воскресния,18.189326,74.0
...,...,...,...
7223,наместнаместник,0.245802,1.0
7224,неистолкованно,0.245802,1.0
7226,уничтожив,0.245802,1.0
7227,умученнаго,0.245802,1.0


## Орфограмма *-аго/-ого*

In [22]:
ago = []
for w in list(delta5df['sort']):
    if re.search('ого$', w):
        ago.append (re.sub(r'ого$', r'аго',w))

ago_df = longer_with_zhy_ia_zm_ot[longer_with_zhy_ia_zm_ot['sort'].isin(ago)]
ago_df['sort'] = ago_df['sort'].apply(lambda w: re.sub(r'аго$', r'ого',w)) #то же самое, но меняем аго на ого


longer_with_zhy_ia_zm_ot_ago = pd.concat([longer_with_zhy_ia_zm_ot, ago_df], sort = False)

delta6df = delta5df[~delta5df['sort'].isin(ago_df['sort'])] #вычитаем из дельты3 5м
delta6df.sort_values(by = 'permil', ascending = False)

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,sort,permil,freq
8546,воскресние,31.708420,129.0
2578,еванглие,29.496205,120.0
4595,раз,27.529791,112.0
6327,аллилуйя,27.283989,111.0
5652,воскресния,18.189326,74.0
...,...,...,...
7221,вшиде,0.245802,1.0
7222,злочастие,0.245802,1.0
7223,наместнаместник,0.245802,1.0
7224,неистолкованно,0.245802,1.0


## Воскресение и Евангелие

Почему-то титлы в сокращениях "воскреСНие (воскресение)" и "ЕванНГЛие" (Евангелие) были неправильно раскрыты в ряде текстов нашего корпуса. Поэтому приходится добавить в расширенный словарь и их тоже. 

In [23]:
voskr = []
for w in list(delta6df['sort']):
    if re.search('воскресн', w):
        voskr.append (re.sub(r'воскресн', r'воскресен',w))

voskr_df = longer_with_zhy_ia_zm_ot_ago[longer_with_zhy_ia_zm_ot_ago['sort'].isin(voskr)]
voskr_df['sort'] = voskr_df['sort'].apply(lambda w: re.sub(r'воскресен', r'воскресн',w)) #то же самое, но меняем аго на ого


longer_with_zhy_ia_zm_ot_ago_voskr = pd.concat([longer_with_zhy_ia_zm_ot_ago, voskr_df], sort = False)

delta7df = delta6df[~delta6df['sort'].isin(voskr_df['sort'])] #вычитаем из дельты3 5м
delta7df.sort_values(by = 'permil', ascending = False)

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,sort,permil,freq
2578,еванглие,29.496205,120.0
4595,раз,27.529791,112.0
6327,аллилуйя,27.283989,111.0
12338,лонгине,17.943525,73.0
4103,филарете,17.206119,70.0
...,...,...,...
7221,вшиде,0.245802,1.0
7222,злочастие,0.245802,1.0
7223,наместнаместник,0.245802,1.0
7224,неистолкованно,0.245802,1.0


In [24]:
evag = []
for w in list(delta7df['sort']):
    if re.search('евангл', w):
        evag.append (re.sub(r'евангл', r'евангел',w))
print(evag)

evag_df = longer_with_zhy_ia_zm_ot_ago_voskr[longer_with_zhy_ia_zm_ot_ago_voskr['sort'].isin(evag)]
evag_df['sort'] = evag_df['sort'].apply(lambda w: re.sub(r'евангел', r'евангл',w)) #то же самое, но меняем аго на ого


longer_with_zhy_ia_zm_ot_ago_voskr_evag = pd.concat([longer_with_zhy_ia_zm_ot_ago_voskr, evag_df], sort = False)

delta8df = delta7df[~delta7df['sort'].isin(evag_df['sort'])] #вычитаем из дельты3 5м
delta8df.sort_values(by = 'permil', ascending = False)

['евангелию', 'евангелии', 'евангелие', 'евангельская', 'евангельский', 'евангельскаго', 'евангелиа', 'евангельскими', 'евангельских', 'евангельски', 'евангельскую', 'евангельскому', 'евангелием']


C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sort,permil,freq
4595,раз,27.529791,112.0
6327,аллилуйя,27.283989,111.0
12338,лонгине,17.943525,73.0
4103,филарете,17.206119,70.0
5586,ноября,10.569473,43.0
...,...,...,...
7220,олег,0.245802,1.0
7221,вшиде,0.245802,1.0
7222,злочастие,0.245802,1.0
7223,наместнаместник,0.245802,1.0


# Теперь надо добавить в словарь парадигмы ещё нескольких слов, по разным причинам отстутствующих в исходном словаре Полякова

Для этого пишем несколько вспомогательных функций. 

`find_freqs` и `find_permils` будет доставать абсолютную и относительную частотность слова из нашего корпуса.

`paragigm_table` будет создавать по начальной форме и парадигме, указанной вручную, все словоформы данного слова и помещать их в датафрейм, легко присоединяемый к существующему удлиненному словарю.

In [25]:
def find_freqs (l, d):
    a=[]
    for w in l:
        try:
            a.append(float(d[d['sort']==w]['freq']))
        except:
            a.append(None)
    return a
print(find_freqs(["поимавый"],delta8df))
def find_permils (l, d):
    a=[]
    for w in l:
        try:
            a.append(float(d[d['sort']==w]['permil']))
        except:
            a.append(None)
    return a
print(find_permils(["поимавый"],delta8df))

[1.0]
[0.24580170684705235]


In [26]:
def paradigm_table(word, paradigm, dictionary):
    worddict = generate_forms(word, paradigm)
    myDict = {key:val for key, val in worddict.items() if not re.match(r'=', val)}
    frqs = find_freqs(list(myDict.values()), delta_df)
    prmls = find_permils(list(myDict.values()), delta_df)
    pddf = pd.DataFrame({"sort": list(map(kill_orthography, list(myDict.values()))),
                        "word": list(myDict.values()),
                        "lex": word,
                        "pos": [None for i in range(len(myDict))],
                        "flex_type": paradigm,
                        "gram": list(myDict.keys()),
                        "pp": [None for i in range(len(myDict))],
                        "freq": frqs,
                        "dup": [None for i in range(len(myDict))],
                        "permil": prmls})
    return pddf

In [27]:
longest = pd.concat([longer_with_zhy_ia_zm_ot_ago_voskr_evag, 
                    paradigm_table("разъ", 'N1t',delta_df),
                    paradigm_table("филаретъ", 'N1t',delta_df),
                    paradigm_table("лонгинъ", "N1t",delta_df),
                    paradigm_table("леонидъ", "N1t",delta_df),
                    paradigm_table("благоюродъ", "N1t",delta_df),
                    paradigm_table("елей", "N1e",delta_df),
                    paradigm_table("согрешити", "V21s",delta_df),
                    paradigm_table("переблаженный", "A1n*",delta_df),
                    paradigm_table("мстиславъ", "N1t",delta_df),
                    paradigm_table("четвергъ", "N1t",delta_df),
                    paradigm_table("углечъ", "N1s",delta_df),
                    
                    paradigm_table("январь", "N1j",delta_df),
                    paradigm_table("февраль", "N1j",delta_df),
                    paradigm_table("мартъ", 'N1t',delta_df),
                    paradigm_table("апрель", "N1j",delta_df),
                    paradigm_table("май", "N1a",delta_df),
                    paradigm_table("июнь", "N1je",delta_df),
                    paradigm_table("июль", "N1je",delta_df),
                    paradigm_table("августъ", 'N1t',delta_df),
                    paradigm_table("сентябрь", 'N1j',delta_df),
                    paradigm_table("октябрь", 'N1j',delta_df),
                    paradigm_table("ноябрь", "N1j",delta_df),
                    paradigm_table("декабрь", "N1j",delta_df),
                     
                    paradigm_table("сноска", 'N3k*-a',delta_df),
                    paradigm_table("домантъ", 'N1t',delta_df),
                    paradigm_table("торжекъ", 'N1t*-a',delta_df),
                    paradigm_table("болгарскiй", 'A1k',delta_df),
                    paradigm_table("рязанскiй", 'A1k',delta_df),
                    paradigm_table("монгольскiй", 'A1k',delta_df),
                    paradigm_table("людоговскiй", 'A1k',delta_df),
                    paradigm_table("русскiй", 'A1k',delta_df),
                    paradigm_table("федоровскiй", 'A1k',delta_df),
                    paradigm_table("рыльскiй", 'A1k',delta_df),
                    paradigm_table("казанскiй", 'A1k',delta_df),
                    paradigm_table("зырянскiй", 'A1k',delta_df),
                    paradigm_table("вологодскiй", 'A1k',delta_df),
                    paradigm_table("ярославскiй", 'A1k',delta_df),
                     
                    paradigm_table("грамматическiй", 'A1k',delta_df),
                    paradigm_table("новоначальный", 'A1t*-b',delta_df),
                    paradigm_table("припѣвати","V11a", delta_df),
                    paradigm_table("жировицкій","A1k*-b", delta_df),
                    paradigm_table("седмочисленникъ", 'N1k',delta_df),
                    paradigm_table("гораздый", 'A1t*-b',delta_df),
                    paradigm_table("оцифровка", 'N3k*-a',delta_df),
                    paradigm_table("тобольскъ", 'N1k',delta_df),
                    paradigm_table("болгаринъ","N1in",delta_df),
                    paradigm_table("царственный", "A1n*",delta_df)
                    ],
                    sort = False)
longest

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,а,##а́,#а##,"S,n,inan",NaN,NaN,NaN,1.0,1,0.223064
1,а,а,а,CONJ,NaN,NaN,NaN,2463.0,1,549.407652
2,ааларь,аала́рь,ааларь,"S,m,anim,persn",N1j,"sg,nom",NaN,1.0,0,0.223064
3,аарона,аарѡ́на,аарон,"S,m,anim,persn",N1t,"sg,gen/acc",NaN,90.0,0,20.075797
4,аароне,аарѡ́не,аарон,"S,m,anim,persn",N1t,"sg,voc",NaN,3.0,0,0.669193
...,...,...,...,...,...,...,...,...,...,...
125,царственейшие,царственѣйшіе,царственный,None,A1n*,"comp1+plen,pl,m,nom1",None,NaN,None,NaN
126,царственне,царственне,царственный,None,A1n*,"brev,sg,m,voc.1",None,NaN,None,NaN
127,царствене,царствене,царственный,None,A1n*,"brev,sg,m,voc.2",None,NaN,None,NaN
128,царственние,царственніе,царственный,None,A1n*,"plen,pl,m,nom1.1",None,NaN,None,NaN


## Некоторые тексты нашего корпуса игнорируют паерок после приставки, отчего в нём есть много слов типа *отём* (отъём)

In [28]:
ot2 = []
for w in list(delta8df['sort']):
    if re.search(r'(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)(я|е|ё|ю)', w):
        ot2.append(re.sub(r'(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)(я|е|ё|ю)',r'\1ъ\2', w))

ot2_df = longer[longer['sort'].isin(ot2)]

ot2_df['sort'] = ot2_df['sort'].apply(lambda w: re.sub(r'^(от|раз|под|пред|без|через|низ|воз|вз|над|чрез|из)ъ(я|е|ё|ю)',r'\1\2', w))

longest = pd.concat([longest,ot2_df], sort = False)
longest

C:\Users\Arseny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,а,##а́,#а##,"S,n,inan",NaN,NaN,NaN,1.0,1,0.223064
1,а,а,а,CONJ,NaN,NaN,NaN,2463.0,1,549.407652
2,ааларь,аала́рь,ааларь,"S,m,anim,persn",N1j,"sg,nom",NaN,1.0,0,0.223064
3,аарона,аарѡ́на,аарон,"S,m,anim,persn",N1t,"sg,gen/acc",NaN,90.0,0,20.075797
4,аароне,аарѡ́не,аарон,"S,m,anim,persn",N1t,"sg,voc",NaN,3.0,0,0.669193
...,...,...,...,...,...,...,...,...,...,...
95991,предят,предъ̑я́тъ,предъяти,"V,pf,tran",V15n,"indic,aor,sg,2p/3p|partcp,praet,pass,brev,sg,m...",NaN,1.0,0,0.223064
109900,разярен,разъ̑яре́нъ,разъярити,"V,pf,tran",V21n,"partcp,praet,pass,brev,sg,m,nom/acc",NaN,1.0,0,0.223064
109901,разярився,разъ̑яри́вся,разъяритися,"V,pf,intr,med",V21n,"partcp,praet,act,brev,sg,m,nom/acc",NaN,2.0,0,0.446129
109902,разярися,разъ̑яри́ся,разъяритися,"V,pf,intr,med",V21n,"imper,sg,2p/3p|indic,aor,sg,2p/3p",NaN,6.0,0,1.338386


In [29]:
delta9df = delta8df[~delta8df['sort'].isin(longest['sort'])]
delta9df.sort_values(by='permil', ascending = False)

,sort,permil,freq
6327,аллилуйя,27.283989,111.0
1903,небесах,8.111456,33.0
19268,панкратие,7.865655,32.0
16029,донате,6.882448,28.0
10584,припеваются,6.636646,27.0
...,...,...,...
7210,склоняюще,0.245802,1.0
7213,состражда,0.245802,1.0
7214,христубогу,0.245802,1.0
7215,делавый,0.245802,1.0


## Добавление близких по расстоянию Левенштейна слов.
В файле `levenshtein_associates.csv` лежат слова нашего корпуса (которых при этом нет в расширенном словаре), расстояние от которых до какого-либо слова, существующего в словаре, равно 1. Если таких слов несколько, берём самое частотное.

In [30]:
new = pd.read_csv('levenshtein_associates.csv', sep = '\t')

In [31]:
one_cognate = new.drop_duplicates(['delta'], keep = False)
one_cognate = one_cognate.rename(columns={"delta": "sort", "polyakov": "word", "lemmas": "lex"})
one_cognate

,sort,word,permil,freq,lex
0,воздигл,воздвигл,0.245802,1.0,воздвигнути
1,богозданна,богоданна,0.245802,1.0,богоданный
42,прогневавшася,прогневашася,0.245802,1.0,прогневатися
50,позвавшему,познавшему,0.491603,2.0,познати
54,путеводными,путеводным,0.245802,1.0,путеводный
...,...,...,...,...,...
31643,екатериной,екатерино,0.245802,1.0,екатерина
31653,сликостоятеля,ликостоятеля,0.245802,1.0,ликостоятель
31670,песньмн,песньми,0.491603,2.0,песнь
31680,посвятивши,освятивши,0.245802,1.0,освятити


In [32]:
longestst = pd.concat([longest, one_cognate], sort = False)
lv_df = new[~new['delta'].isin(one_cognate['sort'])]
lv_df

,delta,polyakov,permil,freq,lemmas
2,врачеванию,врачевание,0.245802,1.0,врачевание
3,врачеванию,врачевании,0.245802,1.0,врачевание
4,врачеванию,врачеваний,0.245802,1.0,врачевание
5,врачеванию,врачевания,0.245802,1.0,врачевание
6,врачеванию,уврачеванию,0.245802,1.0,уврачевание
...,...,...,...,...,...
31675,дастъ,даст,0.491603,2.0,дати
31676,дастъ,дасть,0.491603,2.0,дати
31677,непреподобныя,непреподобная,0.245802,1.0,непреподобный
31678,непреподобныя,непреподобны,0.245802,1.0,непреподобный


In [33]:
one_lex_list = []

for v in set(lv_df['delta']):
    if len(set(lv_df[lv_df['delta']==v]['lemmas']))==1:
        one_lex_list.append(v)
print(len(one_lex_list))

3179


In [34]:
one_lex_df = lv_df[lv_df['delta'].isin(one_lex_list)].drop_duplicates(['delta'], keep = 'first')
one_lex_df = one_lex_df.rename(columns={"delta": "sort", "polyakov": "word", "lemmas": "lex"})
one_lex_df['word'] = [None for i in range (len(one_lex_df['sort']))]
longeststst = pd.concat([longestst, one_lex_df], sort = False, ignore_index = True)
lv_df2 = lv_df[~lv_df['delta'].isin(one_lex_df['sort'])]
lv_df2

,delta,polyakov,permil,freq,lemmas
2,врачеванию,врачевание,0.245802,1.0,врачевание
3,врачеванию,врачевании,0.245802,1.0,врачевание
4,врачеванию,врачеваний,0.245802,1.0,врачевание
5,врачеванию,врачевания,0.245802,1.0,врачевание
6,врачеванию,уврачеванию,0.245802,1.0,уврачевание
...,...,...,...,...,...
31665,пособствовав,пособствова,0.245802,1.0,пособствовати
31666,истнно,истинно,0.245802,1.0,истинный
31667,истнно,истино,0.245802,1.0,истина
31671,вдаются,влаются,0.245802,1.0,влаятися


In [35]:
cumfreq_df = pd.DataFrame(longeststst[longeststst['sort'].isin(set(lv_df2['polyakov']))].groupby('sort').freq.sum())
cumfreq_df['sort'] = cumfreq_df.index
cumfreq_df.index = [i for i in range (len(cumfreq_df['sort']))]
cumfreq_df
lv_df2 = lv_df2.drop(lv_df2[lv_df2['polyakov'].isna()].index)
lv_df2['cumfreq'] = lv_df2['polyakov'].apply(lambda w: float(list(cumfreq_df[cumfreq_df['sort']==w]['freq'])[0]))
lv_df2

,delta,polyakov,permil,freq,lemmas,cumfreq
2,врачеванию,врачевание,0.245802,1.0,врачевание,43.0
3,врачеванию,врачевании,0.245802,1.0,врачевание,6.0
4,врачеванию,врачеваний,0.245802,1.0,врачевание,65.0
5,врачеванию,врачевания,0.245802,1.0,врачевание,46.0
6,врачеванию,уврачеванию,0.245802,1.0,уврачевание,1.0
...,...,...,...,...,...,...
31665,пособствовав,пособствова,0.245802,1.0,пособствовати,6.0
31666,истнно,истинно,0.245802,1.0,истинный,688.0
31667,истнно,истино,0.245802,1.0,истина,2.0
31671,вдаются,влаются,0.245802,1.0,влаятися,1.0


#### Выбираем из всех вариантов интерпретации омонима самый частотный по Полякову:

In [36]:
lv_df2 = lv_df2.drop_duplicates(['delta'], keep = 'first')
lv_df2 = lv_df2.rename(columns={"delta": "sort", "polyakov": "word", "lemmas": "lex"})
lv_df2['word'] = [None for i in range (len(lv_df2['word']))]
del lv_df2['cumfreq']
lv_df2

,sort,word,permil,freq,lex
2,врачеванию,None,0.245802,1.0,врачевание
7,татары,None,0.983207,4.0,тамара
9,ион,None,0.245802,1.0,вон
30,тесныя,None,0.491603,2.0,бесный
38,бденою,None,0.245802,1.0,бдение
...,...,...,...,...,...
31656,приводящею,None,0.245802,1.0,приводити
31660,имеющая,None,0.245802,1.0,имети
31664,пособствовав,None,0.245802,1.0,подобствовати
31666,истнно,None,0.245802,1.0,истинный


In [37]:
longest4 = pd.concat([longeststst, lv_df2], sort = False, ignore_index = True)
longest4

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,а,##а́,#а##,"S,n,inan",NaN,NaN,NaN,1.0,1,0.223064
1,а,а,а,CONJ,NaN,NaN,NaN,2463.0,1,549.407652
2,ааларь,аала́рь,ааларь,"S,m,anim,persn",N1j,"sg,nom",NaN,1.0,0,0.223064
3,аарона,аарѡ́на,аарон,"S,m,anim,persn",N1t,"sg,gen/acc",NaN,90.0,0,20.075797
4,аароне,аарѡ́не,аарон,"S,m,anim,persn",N1t,"sg,voc",NaN,3.0,0,0.669193
...,...,...,...,...,...,...,...,...,...,...
166621,приводящею,None,приводити,NaN,NaN,NaN,NaN,1.0,NaN,0.245802
166622,имеющая,None,имети,NaN,NaN,NaN,NaN,1.0,NaN,0.245802
166623,пособствовав,None,подобствовати,NaN,NaN,NaN,NaN,1.0,NaN,0.245802
166624,истнно,None,истинный,NaN,NaN,NaN,NaN,1.0,NaN,0.245802


In [38]:
lastdelta = delta_df[~delta_df['sort'].isin(longest4['sort'])]
lastdelta.sort_values(by=['permil', 'sort'], ascending = [False, True]).head(50)

,sort,permil,freq
10584,припеваются,6.636646,27.0
3451,диваков,3.441224,14.0
11831,есився,3.441224,14.0
5700,нормам,3.441224,14.0
3333,предначатие,3.441224,14.0
13791,цсязыка,3.441224,14.0
18812,кадисварни,2.703819,11.0
7075,тырнова,2.212215,9.0
2031,феодоровская,2.212215,9.0
12580,арепископа,1.966414,8.0


## На случай, если забыли ещё какие-то важные словоформы:

In [39]:
longest5 = pd.concat([longest4,
                paradigm_table("норма", "N3t",delta_df)],
                    sort = False)



In [40]:
ld = lastdelta[(~lastdelta['sort'].isin(longest5['sort']))].sort_values(by='freq', ascending = False)
ld

,sort,permil,freq
10584,припеваются,6.636646,27.0
11831,есився,3.441224,14.0
3333,предначатие,3.441224,14.0
13791,цсязыка,3.441224,14.0
3451,диваков,3.441224,14.0
...,...,...,...
7080,телотленный,0.245802,1.0
7078,непозыблемому,0.245802,1.0
7061,определяеши,0.245802,1.0
7060,преславнейшую,0.245802,1.0


In [41]:
longest5 = longest5.drop(longest5['sort']=='есив'.index)

In [42]:
splitted_words=[]
for w in tqdm(list(ld['sort'])):
    for i in range (1, len(w)-1):
        if not longest5[longest5['sort']==w[:i]].empty and not longest5[longest5['sort']==w[i:]].empty:
            splitted_words.append([w[:i],w[i:]])


  0%|          | 0/6022 [00:00<?, ?it/s]

In [43]:
len(splitted_words)

3009

## К сожалению, частично слова в корпусе оказались "склеены"

Например, `есився` - это не что иное как "еси вся", где пропущен пробел.

Приходится искусственно их "расклеивать", добавляя в словарь лемматические соответствия типа `sort`: "есився", `lex`: "быти весь".

В то же время, некоторые из кусков этих слов, которые могут быть приняты за самостоятельные слова, на самом деле являются флексиями, по которым можно восстановить лемму.

In [44]:
splw = pd.DataFrame()
for pair in splitted_words:
    if pair[1] in ['го','ми','му','ом','сте','ая','ое','ый','ша','ем','ов']:
        gramdf = longest5[longest5['sort']==pair[0]].sort_values(by='permil', ascending = False)
        newword = pd.DataFrame({"sort": pair[0]+pair[1],
                        "word": None,
                        "lex": list(gramdf['lex'])[0],
                        "pos": list(gramdf['pos'])[0],
                        "flex_type": None,
                        "gram": None,
                        "pp": None,
                        "freq": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['freq']),
                        "dup": None,
                        "permil": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['permil'])}, 
                               index = [0])
        splw = pd.concat([splw, newword], sort = False)
    elif pair[1]=='ся':
        gramdf = longest5[longest5['sort']==pair[0]].sort_values(by='permil', ascending = False)
        newword = pd.DataFrame({"sort": pair[0]+pair[1],
                        "word": None,
                        "lex": list(gramdf['lex'])[0]+pair[1],
                        "pos": list(gramdf['pos'])[0],
                        "flex_type": list(gramdf['flex_type'])[0],
                        "gram": str(list(gramdf['gram'])[0])+',refl.',
                        "pp": None,
                        "freq": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['freq']),
                        "dup": None,
                        "permil": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['permil'])}, 
                               index = [0])
        splw = pd.concat([splw, newword], sort = False)
    elif pair[0] in ['не','со','пред','по','при','на','воз','без','от','из','раз','за','ис','во','под','об','низ','нис','бес','рас']:
        gramdf = longest5[longest5['sort']==pair[1]].sort_values(by='permil', ascending = False)
        newword = pd.DataFrame({"sort": pair[0]+pair[1],
                        "word": None,
                        "lex": pair[0]+list(gramdf['lex'])[0],
                        "pos": list(gramdf['pos'])[0],
                        "flex_type": list(gramdf['flex_type'])[0],
                        "gram": str(list(gramdf['gram'])[0]),
                        "pp": None,
                        "freq": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['freq']),
                        "dup": None,
                        "permil": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['permil'])}, 
                               index = [0])
        splw = pd.concat([splw, newword], sort = False)
    elif pair[0] in ['все','благо','много','присно','перво','добро','велико','зло','миро','священно','сладко','чудо','едино','любо','свято','само','душе','православно','скоро','псково','равно','лже','право','странно','тайно','прекрасно','преподобно','славно','бело','слово','пяти','живо','противо','четверо','три','долго']:
        gramdf = longest5[longest5['sort']==pair[1]].sort_values(by='permil', ascending = False)
        
        newword = pd.DataFrame({"sort": pair[0]+pair[1],
                        "word": None,
                        "lex": pair[0]+list(gramdf['lex'])[0],
                        "pos": list(gramdf['pos'])[0],
                        "flex_type": list(gramdf['flex_type'])[0],
                        "gram": str(list(gramdf['gram'])[0]),
                        "pp": None,
                        "freq": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['freq']),
                        "dup": None,
                        "permil": float(lastdelta[lastdelta['sort']==pair[0]+pair[1]]['permil'])}, 
                               index = [0])
        splw = pd.concat([splw, newword], sort = False)
    elif pair[0] in ['наго','ную','ная','ное','ной','вый','телем']:
        pass
    else:
        pass
splw
        

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,припеваются,None,припеватися,"V,ipf,tran",V11a,"indic,praes,pl,3p,refl.",None,27.0,None,6.636646
0,есився,None,бытися,NaN,NaN,"nan,refl.",None,14.0,None,3.441224
0,исидором,None,исидор,"S,m,anim,persn",None,None,None,8.0,None,1.966414
0,поклонами,None,поклон,"S,m,inan",None,None,None,8.0,None,1.966414
0,победоноснаго,None,победоносный,A,None,None,None,7.0,None,1.720612
...,...,...,...,...,...,...,...,...,...,...
0,недостанет,None,недостати,"V,pf,tran",V15a,"indic,fut,sg,3p",None,1.0,None,0.245802
0,основатися,None,основатися,"V,pf,tran",V12ov/V11a,"inf,refl.",None,1.0,None,0.245802
0,неземную,None,неземный,A,A1t*,"plen,sg,f,acc",None,1.0,None,0.245802
0,смиренномудрому,None,смиренномудрый,NaN,None,None,None,1.0,None,0.245802


In [45]:
splw.head(50)

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,припеваются,None,припеватися,"V,ipf,tran",V11a,"indic,praes,pl,3p,refl.",None,27.0,None,6.636646
0,есився,None,бытися,NaN,NaN,"nan,refl.",None,14.0,None,3.441224
0,исидором,None,исидор,"S,m,anim,persn",None,None,None,8.0,None,1.966414
0,поклонами,None,поклон,"S,m,inan",None,None,None,8.0,None,1.966414
0,победоноснаго,None,победоносный,A,None,None,None,7.0,None,1.720612
0,любострастнаго,None,любострастный,A,A1t*,"plen,sg,m/n,gen",None,6.0,None,1.474810
0,нагосподи,None,нагосподь,"S,m,anim",N1j/N1t,"sg,voc",None,6.0,None,1.474810
0,дерзнувшаго,None,дерзнути,"V,pf,intr",None,None,None,6.0,None,1.474810
0,тропарями,None,тропарь,"S,m,inan",None,None,None,5.0,None,1.229009
0,наусти,None,намстити,NaN,NaN,nan,None,5.0,None,1.229009


In [46]:
longest6 = pd.concat([longest5,splw], sort=False)
longest6.tail(50)

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,отрождшися,None,отродитися,"V,pf,intr,med",V21t,"partcp,praet,act,brev,sg,f,nom/dat/loc|partcp,...",None,1.0,None,0.245802
0,окаянныйая,None,окаянный,A,None,None,None,1.0,None,0.245802
0,священнослужителями,None,священнослужитель,"S,m,anim",N1j,"pl,ins",None,1.0,None,0.245802
0,священнослужителями,None,священнослужитель,NaN,None,None,None,1.0,None,0.245802
0,невиденное,None,невидети,"V,ipf,tran",V22t,"partcp,praet,pass,plen,sg,n,nom/acc",None,1.0,None,0.245802
0,обстоящую,None,обстояти,"V,ipf,intr",V22a,"partcp,praes,act,plen,sg,f,acc",None,1.0,None,0.245802
0,многолюбезен,None,многолюбезный,A,A1t*,"brev,sg,m,nom/acc",None,1.0,None,0.245802
0,сопостящихся,None,сопоститися,"V,ipf,intr,med",V21t,"partcp,praes,act,plen/brev,pl,gen/loc",None,1.0,None,0.245802
0,немирная,None,немирный,A,A1t*,"plen,pl,n,nom/acc|plen,du,m,nom/acc",None,1.0,None,0.245802
0,преумноживша,None,преумножити,NaN,None,None,None,1.0,None,0.245802


In [47]:
lld = ld[(~ld['sort'].isin(longest6['sort']))]
lld.sort_values(by='freq', ascending = False)#.head(50)

,sort,permil,freq
3333,предначатие,3.441224,14.0
3451,диваков,3.441224,14.0
13791,цсязыка,3.441224,14.0
18812,кадисварни,2.703819,11.0
2031,феодоровская,2.212215,9.0
...,...,...,...
18768,идолопоклонных,0.245802,1.0
18766,ангелвозненавидевши,0.245802,1.0
18764,выполнит,0.245802,1.0
18761,есизде,0.245802,1.0


In [48]:
possible_spl = list(set([w[0]+w[1] for w in splitted_words])-set(list(splw['sort'])))

In [49]:
def split_non_affixed(w):
    interpretations = []
    for i in range (1, len(w)-1):
        if not longest5[longest5['sort']==w[:i]].empty and not longest5[longest5['sort']==w[i:]].empty:
            word1df = longest5[longest5['sort']==w[:i]].sort_values(by='permil', ascending = False)
            word2df = longest5[longest5['sort']==w[i:]].sort_values(by='permil', ascending = False)
            interpretation_dic = {"sort": w[:i]+w[i:], 
                                  "word": None,
                                  "lex": list(word1df['lex'])[0] + " " + list(word2df['lex'])[0],
                                  "pos": None,
                                  "flex_type": None,
                                  "gram": None,
                                  "pp": None,
                                  "freq": float(list(word1df['freq'])[0])+float(list(word2df['freq'])[0]),
                                  "dup": None,
                                  "permil": float(list(word1df['permil'])[0])+float(list(word2df['permil'])[0])
                               }
            interpretations.append(interpretation_dic)
    if len(interpretations)>0:
        return sorted(interpretations, key=lambda x: x['freq'], reverse=True)[0]
    else:
        return None

In [50]:
splitted_non_affixed = []
for w in tqdm(possible_spl):
    splitted_non_affixed.append(split_non_affixed(w))

pd.DataFrame(filter(None, splitted_non_affixed)).head(50)

  0%|          | 0/1453 [00:00<?, ?it/s]

,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
0,благовестившия,None,благовестити ваиа,None,None,None,None,20.0,None,4.484026
1,деснымосужденнаго,None,десный осудити,None,None,None,None,40.0,None,8.922577
2,братоубийцею,None,братоубийца и,None,None,None,None,92.0,None,20.521926
3,обожаемьпищи,None,обожати пища,None,None,None,None,385.0,None,85.879800
4,воцаривши,None,воцарити бити,None,None,None,None,4.0,None,0.983207
5,духородне,None,дух родне,None,None,None,None,2.0,None,0.468866
6,псалместихира,None,псалом стихира,None,None,None,None,1571.0,None,350.434195
7,непреоборимии,None,непреоборимый и҃и,None,None,None,None,620.0,None,138.345411
8,премудренетлению,None,премудрый нетление,None,None,None,None,503.0,None,112.201400
9,богомудреревнуя,None,богомудрый ревновати,None,None,None,None,971.0,None,216.595546


In [51]:
longest7 = pd.concat([longest6, pd.DataFrame(filter(None, splitted_non_affixed))])

## Получившийся словарь записываем в файл `new_full_vocabulary.csv`.

Он отсортирован по частотности. Теперь для каждой словоформы достаточно найти первое соответствие в словаре, и, с некоторым количеством ошибок, лемматизатор будет работать.

Неопознанными остались 3335 слов корпуса, в основном из метатекста.

In [52]:
longest7.sort_values(by='permil', ascending=False).to_csv('new_full_vocabulary.csv', sep = '\t')


In [53]:
llld = lld[(~lld['sort'].isin(longest7['sort']))]
llld

,sort,permil,freq
3333,предначатие,3.441224,14.0
13791,цсязыка,3.441224,14.0
3451,диваков,3.441224,14.0
2031,феодоровская,2.212215,9.0
7075,тырнова,2.212215,9.0
...,...,...,...
7092,сладколюбящею,0.245802,1.0
7089,доброчадная,0.245802,1.0
7083,ювеналия,0.245802,1.0
7061,определяеши,0.245802,1.0


## Словарь только омонимов

In [54]:
os.chdir('C:\\Users\\Arseny\\Desktop\\repository\\ocs')
d = pd.DataFrame()
v =  pd.read_csv('voc.csv', sep = '\t')
v = v.drop_duplicates(['sort', 'lex'], keep = 'first')
omonyms = v[v.duplicated(['sort'], keep = False)]

In [55]:
omonyms.head(60)
omonyms.to_csv('omonyms.csv',sep='\t')

In [56]:
omonyms.sort_values(by='sort').tail(60)

,Unnamed: 0,sort,word,lex,pos,flex_type,gram,pp,freq,dup,permil
35158,144653,чинно,чи́ннѡ,чинно,ADV,NaN,NaN,NaN,7.0,1.0,1.561451
111536,144652,чинно,чи́нно,чинный,A,A1t*,"brev,sg,n,nom/acc",NaN,1.0,1.0,0.223064
59181,145119,чисти,чс̑ти,честь++,"S,f,inan",N41,"sg,gen/dat/loc",NaN,3.0,0.0,0.669193
11737,144749,чисти,чи́сти,чистый,A,A1t,"brev,pl,m,nom",NaN,38.0,1.0,8.476448
5242,144770,чисто,чи́стѡ,чисто,ADV,NaN,NaN,NaN,115.0,1.0,25.652408
3463,144769,чисто,чи́сто,чистый,A,A1t,"brev,sg,n,nom/acc",NaN,200.0,1.0,44.612883
34430,145346,чувственно,чу́вственно,чувственный,A,A1n*,"brev,sg,n,nom/acc",NaN,8.0,1.0,1.784515
17346,145347,чувственно,чу́вственнѡ,чувственно,ADV,NaN,NaN,NaN,22.0,1.0,4.907417
22814,145419,чудесне,чуде́снѣ,чудесне,ADV,NaN,NaN,NaN,14.0,0.0,3.122902
112282,145418,чудесне,чуде́сне,чудесный,A,A1t*,"brev,sg,m,voc",NaN,1.0,0.0,0.223064
